In [1]:

import pandas as pd
import ctypes

# F1 2021 UDP Specificaion

### Packet Header

In [ ]:
class PacketHeader(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_packet_format', ctypes.c_uint16), # 2021
        ('m_game_major_version', ctypes.c_uint8), # Game major version - "X.00"
        ('m_game_minor_version', ctypes.c_uint8), # Game minor version - "1.XX"
        ('m_packet_version', ctypes.c_uint8), # Version of this packet type, all start from 1
        ('m_packet_id', ctypes.c_uint8), # Identifier for the packet type, see below
        ('m_session_uid', ctypes.c_uint64), # Unique identifier for the session
        ('m_session_time', ctypes.c_float), # Session timestamp
        ('m_frame_identifier', ctypes.c_uint32), # Identifier for the frame the data was retrieved on
        ('m_player_car_index', ctypes.c_uint8), # Index of player's car in the array
        ('m_secondary_player_car_index', ctypes.c_uint8), # Index of secondary player's car in the array (splitscreen)
        # 255 if no second player
    ]


### 0 - Motion
Packet Size: 1464

In [2]:
# ===========================================================   PACKET ID 0: MOTION   ============================================================
class CarMotionData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_world_position_x', ctypes.c_float), # World space X position
        ('m_world_position_y', ctypes.c_float), # World space Y position
        ('m_world_position_z', ctypes.c_float), # World space Z position
        ('m_world_velocity_x', ctypes.c_float), # Velocity in world space X
        ('m_world_velocity_y', ctypes.c_float), # Velocity in world space Y
        ('m_world_velocity_z', ctypes.c_float), # Velocity in world space Z
        ('m_world_forward_dir_x', ctypes.c_int16), # World space forward X direction (normalised)
        ('m_world_forward_dir_y', ctypes.c_int16), # World space forward Y direction (normalised)
        ('m_world_forward_dir_z', ctypes.c_int16), # World space forward Z direction (normalised)
        ('m_world_right_dir_x', ctypes.c_int16), # World space right X direction (normalised)
        ('m_world_right_dir_y', ctypes.c_int16), # World space right Y direction (normalised)
        ('m_world_right_dir_z', ctypes.c_int16), # World space right Z direction (normalised)
        ('m_g_force_lateral', ctypes.c_float), # Lateral G-Force component
        ('m_g_force_longitudinal', ctypes.c_float), # Longitudinal G-Force component
        ('m_g_force_vertical', ctypes.c_float), # Vertical G-Force component
        ('m_yaw', ctypes.c_float), # Yaw angle in radians
        ('m_pitch', ctypes.c_float), # Pitch angle in radians
        ('m_roll', ctypes.c_float), # Roll angle in radians
    ]

class PacketMotionData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_header', PacketHeader), # Header
        ('m_car_motion_data', CarMotionData * 22), # Data for all cars on track        # Extra player car ONLY data
        ('m_suspension_position', ctypes.c_float * 4), # Note: All wheel arrays have the following order:
        ('m_suspension_velocity', ctypes.c_float * 4), # RL, RR, FL, FR
        ('m_suspension_acceleration', ctypes.c_float * 4), # RL, RR, FL, FR
        ('m_wheel_speed', ctypes.c_float * 4), # Speed of each wheel
        ('m_wheel_slip', ctypes.c_float * 4), # Slip ratio for each wheel
        ('m_local_velocity_x', ctypes.c_float), # Velocity in local space
        ('m_local_velocity_y', ctypes.c_float), # Velocity in local space
        ('m_local_velocity_z', ctypes.c_float), # Velocity in local space
        ('m_angular_velocity_x', ctypes.c_float), # Angular velocity x-component
        ('m_angular_velocity_y', ctypes.c_float), # Angular velocity y-component
        ('m_angular_velocity_z', ctypes.c_float), # Angular velocity z-component
        ('m_angular_acceleration_x', ctypes.c_float), # Angular velocity x-component
        ('m_angular_acceleration_y', ctypes.c_float), # Angular velocity y-component
        ('m_angular_acceleration_z', ctypes.c_float), # Angular velocity z-component
        ('m_front_wheels_angle', ctypes.c_float), # Current front wheels angle in radians
    ]


### 1 - Session
Packet Size: 625

In [ ]:
# ===========================================================   PACKET ID 1: SESSION  ============================================================
class MarshalZone(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_zone_start', ctypes.c_float), # Fraction (0..1) of way through the lap the marshal zone starts
        ('m_zone_flag', ctypes.c_int8), # -1 = invalid/unknown, 0 = none, 1 = green, 2 = blue, 3 = yellow, 4 = red
    ]

class WeatherForecastSample(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_session_type', ctypes.c_uint8), # 0 = unknown, 1 = P1, 2 = P2, 3 = P3, 4 = Short P, 5 = Q1
        # 6 = Q2, 7 = Q3, 8 = Short Q, 9 = OSQ, 10 = R, 11 = R2
        # 12 = Time Trial
        ('m_time_offset', ctypes.c_uint8), # Time in minutes the forecast is for
        ('m_weather', ctypes.c_uint8), # Weather - 0 = clear, 1 = light cloud, 2 = overcast
        # 3 = light rain, 4 = heavy rain, 5 = storm
        ('m_track_temperature', ctypes.c_int8), # Track temp. in degrees Celsius
        ('m_track_temperature_change', ctypes.c_int8), # Track temp. change – 0 = up, 1 = down, 2 = no change
        ('m_air_temperature', ctypes.c_int8), # Air temp. in degrees celsius
        ('m_air_temperature_change', ctypes.c_int8), # Air temp. change – 0 = up, 1 = down, 2 = no change
        ('m_rain_percentage', ctypes.c_uint8), # Rain percentage (0-100)
    ]


class PacketSessionData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_header', PacketHeader), # Header
        ('m_weather', ctypes.c_uint8), # Weather - 0 = clear, 1 = light cloud, 2 = overcast        # 3 = light rain, 4 = heavy rain, 5 = storm
        ('m_track_temperature', ctypes.c_int8), # Track temp. in degrees celsius
        ('m_air_temperature', ctypes.c_int8), # Air temp. in degrees celsius
        ('m_total_laps', ctypes.c_uint8), # Total number of laps in this race
        ('m_track_length', ctypes.c_uint16), # Track length in metres
        ('m_session_type', ctypes.c_uint8), # 0 = unknown, 1 = P1, 2 = P2, 3 = P3, 4 = Short P        # 5 = Q1, 6 = Q2, 7 = Q3, 8 = Short Q, 9 = OSQ        # 10 = R, 11 = R2, 12 = R3, 13 = Time Trial
        ('m_track_id', ctypes.c_int8), # -1 for unknown, 0-21 for tracks, see appendix
        ('m_formula', ctypes.c_uint8), # Formula, 0 = F1 Modern, 1 = F1 Classic, 2 = F2,        # 3 = F1 Generic
        ('m_session_time_left', ctypes.c_uint16), # Time left in session in seconds
        ('m_session_duration', ctypes.c_uint16), # Session duration in seconds
        ('m_pit_speed_limit', ctypes.c_uint8), # Pit speed limit in kilometres per hour
        ('m_game_paused', ctypes.c_uint8), # Whether the game is paused
        ('m_is_spectating', ctypes.c_uint8), # Whether the player is spectating
        ('m_spectator_car_index', ctypes.c_uint8), # Index of the car being spectated
        ('m_sli_pro_native_support', ctypes.c_uint8), # SLI Pro support, 0 = inactive, 1 = active
        ('m_num_marshal_zones', ctypes.c_uint8), # Number of marshal zones to follow
        ('m_marshal_zones', MarshalZone * 21), # List of marshal zones – max 21
        ('m_safety_car_status', ctypes.c_uint8), # 0 = no safety car, 1 = full        # 2 = virtual, 3 = formation lap
        ('m_network_game', ctypes.c_uint8), # 0 = offline, 1 = online
        ('m_num_weather_forecast_samples', ctypes.c_uint8), # Number of weather samples to follow
        ('m_weather_forecast_samples', WeatherForecastSample * 56), # Array of weather forecast samples
        ('m_forecast_accuracy', ctypes.c_uint8), # 0 = Perfect, 1 = Approximate
        ('m_ai_difficulty', ctypes.c_uint8), # AI Difficulty rating – 0-110
        ('m_season_link_identifier', ctypes.c_uint32), # Identifier for season - persists across saves
        ('m_weekend_link_identifier', ctypes.c_uint32), # Identifier for weekend - persists across saves
        ('m_session_link_identifier', ctypes.c_uint32), # Identifier for session - persists across saves
        ('m_pit_stop_window_ideal_lap', ctypes.c_uint8), # Ideal lap to pit on for current strategy (player)
        ('m_pit_stop_window_latest_lap', ctypes.c_uint8), # Latest lap to pit on for current strategy (player)
        ('m_pit_stop_rejoin_position', ctypes.c_uint8), # Predicted position to rejoin at (player)
        ('m_steering_assist', ctypes.c_uint8), # 0 = off, 1 = on
        ('m_braking_assist', ctypes.c_uint8), # 0 = off, 1 = low, 2 = medium, 3 = high
        ('m_gearbox_assist', ctypes.c_uint8), # 1 = manual, 2 = manual & suggested gear, 3 = auto
        ('m_pit_assist', ctypes.c_uint8), # 0 = off, 1 = on
        ('m_pit_release_assist', ctypes.c_uint8), # 0 = off, 1 = on
        ('m_ersassist', ctypes.c_uint8), # 0 = off, 1 = on
        ('m_drsassist', ctypes.c_uint8), # 0 = off, 1 = on
        ('m_dynamic_racing_line', ctypes.c_uint8), # 0 = off, 1 = corners only, 2 = full
        ('m_dynamic_racing_line_type', ctypes.c_uint8), # 0 = 2D, 1 = 3D
    ]


In [3]:

class FastestLap(ctypes.LittleEndianStructure):
    _fields_ = [
        ('vehicle_idx', ctypes.c_uint8), # Vehicle index of car achieving fastest lap
        ('lap_time', ctypes.c_float), # Lap time is in seconds
    ]

class Retirement(ctypes.LittleEndianStructure):
    _fields_ = [
        ('vehicle_idx', ctypes.c_uint8), # Vehicle index of car retiring
    ]

class TeamMateInPits(ctypes.LittleEndianStructure):
    _fields_ = [
        ('vehicle_idx', ctypes.c_uint8), # Vehicle index of team mate
    ]

class RaceWinner(ctypes.LittleEndianStructure):
    _fields_ = [
        ('vehicle_idx', ctypes.c_uint8), # Vehicle index of the race winner
    ]

class Penalty(ctypes.LittleEndianStructure):
    _fields_ = [
        ('penalty_type', ctypes.c_uint8), # Penalty type – see Appendices
        ('infringement_type', ctypes.c_uint8), # Infringement type – see Appendices
        ('vehicle_idx', ctypes.c_uint8), # Vehicle index of the car the penalty is applied to
        ('other_vehicle_idx', ctypes.c_uint8), # Vehicle index of the other car involved
        ('time', ctypes.c_uint8), # Time gained, or time spent doing action in seconds
        ('lap_num', ctypes.c_uint8), # Lap the penalty occurred on
        ('places_gained', ctypes.c_uint8), # Number of places gained by this
    ]

class SpeedTrap(ctypes.LittleEndianStructure):
    _fields_ = [
        ('vehicle_idx', ctypes.c_uint8), # Vehicle index of the vehicle triggering speed trap
        ('speed', ctypes.c_float), # Top speed achieved in kilometres per hour
        ('overall_fastest_in_session', ctypes.c_uint8), # Overall fastest speed in session = 1, otherwise 0
        ('driver_fastest_in_session', ctypes.c_uint8), # Fastest speed for driver in session = 1, otherwise 0
    ]

class StartLIghts(ctypes.LittleEndianStructure):
    _fields_ = [
        ('num_lights', ctypes.c_uint8), # Number of lights showing
    ]

class DriveThroughPenaltyServed(ctypes.LittleEndianStructure):
    _fields_ = [
        ('vehicle_idx', ctypes.c_uint8), # Vehicle index of the vehicle serving drive through
    ]

class StopGoPenaltyServed(ctypes.LittleEndianStructure):
    _fields_ = [
        ('vehicle_idx', ctypes.c_uint8), # Vehicle index of the vehicle serving stop go
    ]

class Flashback(ctypes.LittleEndianStructure):
    _fields_ = [
        ('flashback_frame_identifier', ctypes.c_uint32), # Frame identifier flashed back to
        ('flashback_session_time', ctypes.c_float), # Session time flashed back to
    ]

class Buttons(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_button_status', ctypes.c_uint32), # Bit flags specifying which buttons are being pressed
        # currently - see appendices
    ]

class PacketEventData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_header', PacketHeader), # Header
        ('m_event_string_code', ctypes.c_uint8 * 4), # Event string code, see below
        ('m_event_details', PacketSessionData), # Event details - should be interpreted differently
        # for each type
    ]

class ParticipantData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_ai_controlled', ctypes.c_uint8), # Whether the vehicle is AI (1) or Human (0) controlled
        ('m_driver_id', ctypes.c_uint8), # Driver id - see appendix, 255 if network human
        ('m_network_id', ctypes.c_uint8), # Network id – unique identifier for network players
        ('m_team_id', ctypes.c_uint8), # Team id - see appendix
        ('m_my_team', ctypes.c_uint8), # My team flag – 1 = My Team, 0 = otherwise
        ('m_race_number', ctypes.c_uint8), # Race number of the car
        ('m_nationality', ctypes.c_uint8), # Nationality of the driver
        ('m_name', ctypes.c_char * 48), # Name of participant in UTF-8 format – null terminated
        # Will be truncated with … (U+2026) if too long
        ('m_your_telemetry', ctypes.c_uint8), # The player's UDP setting, 0 = restricted, 1 = public
    ]

class PacketParticipantsData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_header', PacketHeader), # Header
        ('m_num_active_cars', ctypes.c_uint8), # Number of active cars in the data – should match number of
        # cars on HUD
        ('m_participants', ParticipantData * 22),
    ]

class CarSetupData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_front_wing', ctypes.c_uint8), # Front wing aero
        ('m_rear_wing', ctypes.c_uint8), # Rear wing aero
        ('m_on_throttle', ctypes.c_uint8), # Differential adjustment on throttle (percentage)
        ('m_off_throttle', ctypes.c_uint8), # Differential adjustment off throttle (percentage)
        ('m_front_camber', ctypes.c_float), # Front camber angle (suspension geometry)
        ('m_rear_camber', ctypes.c_float), # Rear camber angle (suspension geometry)
        ('m_front_toe', ctypes.c_float), # Front toe angle (suspension geometry)
        ('m_rear_toe', ctypes.c_float), # Rear toe angle (suspension geometry)
        ('m_front_suspension', ctypes.c_uint8), # Front suspension
        ('m_rear_suspension', ctypes.c_uint8), # Rear suspension
        ('m_front_anti_roll_bar', ctypes.c_uint8), # Front anti-roll bar
        ('m_rear_anti_roll_bar', ctypes.c_uint8), # Front anti-roll bar
        ('m_front_suspension_height', ctypes.c_uint8), # Front ride height
        ('m_rear_suspension_height', ctypes.c_uint8), # Rear ride height
        ('m_brake_pressure', ctypes.c_uint8), # Brake pressure (percentage)
        ('m_brake_bias', ctypes.c_uint8), # Brake bias (percentage)
        ('m_rear_left_tyre_pressure', ctypes.c_float), # Rear left tyre pressure (PSI)
        ('m_rear_right_tyre_pressure', ctypes.c_float), # Rear right tyre pressure (PSI)
        ('m_front_left_tyre_pressure', ctypes.c_float), # Front left tyre pressure (PSI)
        ('m_front_right_tyre_pressure', ctypes.c_float), # Front right tyre pressure (PSI)
        ('m_ballast', ctypes.c_uint8), # Ballast
        ('m_fuel_load', ctypes.c_float), # Fuel load
    ]

class PacketCarSetupData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_header', PacketHeader), # Header
        ('m_car_setups', CarSetupData * 22),
    ]

class CarTelemetryData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_speed', ctypes.c_uint16), # Speed of car in kilometres per hour
        ('m_throttle', ctypes.c_float), # Amount of throttle applied (0.0 to 1.0)
        ('m_steer', ctypes.c_float), # Steering (-1.0 (full lock left) to 1.0 (full lock right))
        ('m_brake', ctypes.c_float), # Amount of brake applied (0.0 to 1.0)
        ('m_clutch', ctypes.c_uint8), # Amount of clutch applied (0 to 100)
        ('m_gear', ctypes.c_int8), # Gear selected (1-8, N=0, R=-1)
        ('m_engine_rpm', ctypes.c_uint16), # Engine RPM
        ('m_drs', ctypes.c_uint8), # 0 = off, 1 = on
        ('m_rev_lights_percent', ctypes.c_uint8), # Rev lights indicator (percentage)
        ('m_rev_lights_bit_value', ctypes.c_uint16), # Rev lights (bit 0 = leftmost LED, bit 14 = rightmost LED)
        ('m_brakes_temperature', ctypes.c_uint16 * 4), # Brakes temperature (celsius)
        ('m_tyres_surface_temperature', ctypes.c_uint8 * 4), # Tyres surface temperature (celsius)
        ('m_tyres_inner_temperature', ctypes.c_uint8 * 4), # Tyres inner temperature (celsius)
        ('m_engine_temperature', ctypes.c_uint16), # Engine temperature (celsius)
        ('m_tyres_pressure', ctypes.c_float * 4), # Tyres pressure (PSI)
        ('m_surface_type', ctypes.c_uint8 * 4), # Driving surface, see appendices
    ]

class PacketCarTelemetryData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_header', PacketHeader), # Header
        ('m_car_telemetry_data', CarTelemetryData * 22),
        ('m_mfd_panel_index', ctypes.c_uint8), # Index of MFD panel open - 255 = MFD closed
        # Single player, race – 0 = Car setup, 1 = Pits
        # 2 = Damage, 3 =  Engine, 4 = Temperatures
        # May vary depending on game mode
        ('m_mfd_panel_index_secondary_player', ctypes.c_uint8), # See above
        ('m_suggested_gear', ctypes.c_int8), # Suggested gear for the player (1-8)
        # 0 if no gear suggested
    ]

class CarStatusData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_traction_control', ctypes.c_uint8), # Traction control - 0 = off, 1 = medium, 2 = full
        ('m_anti_lock_brakes', ctypes.c_uint8), # 0 (off) - 1 (on)
        ('m_fuel_mix', ctypes.c_uint8), # Fuel mix - 0 = lean, 1 = standard, 2 = rich, 3 = max
        ('m_front_brake_bias', ctypes.c_uint8), # Front brake bias (percentage)
        ('m_pit_limiter_status', ctypes.c_uint8), # Pit limiter status - 0 = off, 1 = on
        ('m_fuel_in_tank', ctypes.c_float), # Current fuel mass
        ('m_fuel_capacity', ctypes.c_float), # Fuel capacity
        ('m_fuel_remaining_laps', ctypes.c_float), # Fuel remaining in terms of laps (value on MFD)
        ('m_max_rpm', ctypes.c_uint16), # Cars max RPM, point of rev limiter
        ('m_idle_rpm', ctypes.c_uint16), # Cars idle RPM
        ('m_max_gears', ctypes.c_uint8), # Maximum number of gears
        ('m_drs_allowed', ctypes.c_uint8), # 0 = not allowed, 1 = allowed
        ('m_drs_activation_distance', ctypes.c_uint16), # 0 = DRS not available, non-zero - DRS will be available        # in [X] metres
        ('m_actual_tyre_compound', ctypes.c_uint8), # F1 Modern - 16 = C5, 17 = C4, 18 = C3, 19 = C2, 20 = C1        # 7 = inter, 8 = wet        # F1 Classic - 9 = dry, 10 = wet        # F2 – 11 = super soft, 12 = soft, 13 = medium, 14 = hard        # 15 = wet
        ('m_visual_tyre_compound', ctypes.c_uint8), # F1 visual (can be different from actual compound)        # 16 = soft, 17 = medium, 18 = hard, 7 = inter, 8 = wet        # F1 Classic – same as above        # F2 ‘19, 15 = wet, 19 – super soft, 20 = soft        # 21 = medium , 22 = hard
        ('m_tyres_age_laps', ctypes.c_uint8), # Age in laps of the current set of tyres
        ('m_vehicle_fia_flags', ctypes.c_int8), # -1 = invalid/unknown, 0 = none, 1 = green        # 2 = blue, 3 = yellow, 4 = red
        ('m_ers_store_energy', ctypes.c_float), # ERS energy store in Joules
        ('m_ers_deploy_mode', ctypes.c_uint8), # ERS deployment mode, 0 = none, 1 = medium        # 2 = hotlap, 3 = overtake
        ('m_ers_harvested_this_lap_mguk', ctypes.c_float), # ERS energy harvested this lap by MGU-K
        ('m_ers_harvested_this_lap_mguh', ctypes.c_float), # ERS energy harvested this lap by MGU-H
        ('m_ers_deployed_this_lap', ctypes.c_float), # ERS energy deployed this lap
        ('m_network_paused', ctypes.c_uint8), # Whether the car is paused in a network game
    ]

class PacketCarStatusData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_header', PacketHeader), # Header
        ('m_car_status_data', CarStatusData * 22),
    ]

class FinalClassificationData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_position', ctypes.c_uint8), # Finishing position
        ('m_num_laps', ctypes.c_uint8), # Number of laps completed
        ('m_grid_position', ctypes.c_uint8), # Grid position of the car
        ('m_points', ctypes.c_uint8), # Number of points scored
        ('m_num_pit_stops', ctypes.c_uint8), # Number of pit stops made
        ('m_result_status', ctypes.c_uint8), # Result status - 0 = invalid, 1 = inactive, 2 = active
        # 3 = finished, 4 = didnotfinish, 5 = disqualified
        # 6 = not classified, 7 = retired
        ('m_best_lap_time_in_ms', ctypes.c_uint32), # Best lap time of the session in milliseconds
        ('m_total_race_time', ctypes.c_double), # Total race time in seconds without penalties
        ('m_penalties_time', ctypes.c_uint8), # Total penalties accumulated in seconds
        ('m_num_penalties', ctypes.c_uint8), # Number of penalties applied to this driver
        ('m_num_tyre_stints', ctypes.c_uint8), # Number of tyres stints up to maximum
        ('m_tyre_stints_actual', ctypes.c_uint8 * 8), # Actual tyres used by this driver
        ('m_tyre_stints_visual', ctypes.c_uint8 * 8), # Visual tyres used by this driver
    ]

class PacketFinalClassificationData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_header', PacketHeader), # Header
        ('m_num_cars', ctypes.c_uint8), # Number of cars in the final classification
        ('m_classification_data', FinalClassificationData * 22),
    ]

class LobbyInfoData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_ai_controlled', ctypes.c_uint8), # Whether the vehicle is AI (1) or Human (0) controlled
        ('m_team_id', ctypes.c_uint8), # Team id - see appendix (255 if no team currently selected)
        ('m_nationality', ctypes.c_uint8), # Nationality of the driver
        ('m_name', ctypes.c_char * 48), # Name of participant in UTF-8 format – null terminated
        # Will be truncated with ... (U+2026) if too long
        ('m_car_number', ctypes.c_uint8), # Car number of the player
        ('m_ready_status', ctypes.c_uint8), # 0 = not ready, 1 = ready, 2 = spectating
    ]

class PacketLobbyInfoData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_header', PacketHeader), # Header
        # Packet specific data
        ('m_num_players', ctypes.c_uint8), # Number of players in the lobby data
        ('m_lobby_players', LobbyInfoData * 22),
    ]

class CarDamageData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_tyres_wear', ctypes.c_float * 4), # Tyre wear (percentage)
        ('m_tyres_damage', ctypes.c_uint8 * 4), # Tyre damage (percentage)
        ('m_brakes_damage', ctypes.c_uint8 * 4), # Brakes damage (percentage)
        ('m_front_left_wing_damage', ctypes.c_uint8), # Front left wing damage (percentage)
        ('m_front_right_wing_damage', ctypes.c_uint8), # Front right wing damage (percentage)
        ('m_rear_wing_damage', ctypes.c_uint8), # Rear wing damage (percentage)
        ('m_floor_damage', ctypes.c_uint8), # Floor damage (percentage)
        ('m_diffuser_damage', ctypes.c_uint8), # Diffuser damage (percentage)
        ('m_sidepod_damage', ctypes.c_uint8), # Sidepod damage (percentage)
        ('m_drs_fault', ctypes.c_uint8), # Indicator for DRS fault, 0 = OK, 1 = fault
        ('m_gear_box_damage', ctypes.c_uint8), # Gear box damage (percentage)
        ('m_engine_damage', ctypes.c_uint8), # Engine damage (percentage)
        ('m_engine_mguhwear', ctypes.c_uint8), # Engine wear MGU-H (percentage)
        ('m_engine_eswear', ctypes.c_uint8), # Engine wear ES (percentage)
        ('m_engine_cewear', ctypes.c_uint8), # Engine wear CE (percentage)
        ('m_engine_icewear', ctypes.c_uint8), # Engine wear ICE (percentage)
        ('m_engine_mgukwear', ctypes.c_uint8), # Engine wear MGU-K (percentage)
        ('m_engine_tcwear', ctypes.c_uint8), # Engine wear TC (percentage)
    ]

class PacketCarDamageData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_header', PacketHeader), # Header
        ('m_car_damage_data', CarDamageData * 22),
    ]

class LapHistoryData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_lap_time_in_ms', ctypes.c_uint32), # Lap time in milliseconds
        ('m_sector1_time_in_ms', ctypes.c_uint16), # Sector 1 time in milliseconds
        ('m_sector2_time_in_ms', ctypes.c_uint16), # Sector 2 time in milliseconds
        ('m_sector3_time_in_ms', ctypes.c_uint16), # Sector 3 time in milliseconds
        ('m_lap_valid_bit_flags', ctypes.c_uint8), # 0x01 bit set-lap valid,      0x02 bit set-sector 1 valid
        # 0x04 bit set-sector 2 valid, 0x08 bit set-sector 3 valid
    ]

class TyreStintHistoryData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_end_lap', ctypes.c_uint8), # Lap the tyre usage ends on (255 of current tyre)
        ('m_tyre_actual_compound', ctypes.c_uint8), # Actual tyres used by this driver
        ('m_tyre_visual_compound', ctypes.c_uint8), # Visual tyres used by this driver
    ]

class PacketSessionHistoryData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_header', PacketHeader), # Header
        ('m_car_idx', ctypes.c_uint8), # Index of the car this lap data relates to
        ('m_num_laps', ctypes.c_uint8), # Num laps in the data (including current partial lap)
        ('m_num_tyre_stints', ctypes.c_uint8), # Number of tyre stints in the data
        ('m_best_lap_time_lap_num', ctypes.c_uint8), # Lap the best lap time was achieved on
        ('m_best_sector1_lap_num', ctypes.c_uint8), # Lap the best Sector 1 time was achieved on
        ('m_best_sector2_lap_num', ctypes.c_uint8), # Lap the best Sector 2 time was achieved on
        ('m_best_sector3_lap_num', ctypes.c_uint8), # Lap the best Sector 3 time was achieved on
        ('m_lap_history_data', LapHistoryData * 100), # 100 laps of data max
        ('m_tyre_stints_history_data', TyreStintHistoryData * 8),
    ]
 



 

# class Test_PacketHeader(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_packet_format', ctypes.c_uint16), # 2021
#             ('m_game_major_version', ctypes.c_uint8), # Game major version - "X.00"
#             ('m_game_minor_version', ctypes.c_uint8), # Game minor version - "1.XX"
#             ('m_packet_version', ctypes.c_uint8), # Version of this packet type, all start from 1
#             ('m_packet_id', ctypes.c_uint8), # Identifier for the packet type, see below
#             ('m_session_uid', ctypes.c_uint64), # Unique identifier for the session
#             ('m_session_time', ctypes.c_float), # Session timestamp
#             ('m_frame_identifier', ctypes.c_uint32), # Identifier for the frame the data was retrieved on
#             ('m_player_car_index', ctypes.c_uint8), # Index of player's car in the array
#             ('m_secondary_player_car_index', ctypes.c_uint8), # Index of secondary player's car in the array (splitscreen)
#             # 255 if no second player
#         ]
#     )
#     def test_PacketHeader__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = PacketHeader()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_CarMotionData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_world_position_x', ctypes.c_float), # World space X position
#             ('m_world_position_y', ctypes.c_float), # World space Y position
#             ('m_world_position_z', ctypes.c_float), # World space Z position
#             ('m_world_velocity_x', ctypes.c_float), # Velocity in world space X
#             ('m_world_velocity_y', ctypes.c_float), # Velocity in world space Y
#             ('m_world_velocity_z', ctypes.c_float), # Velocity in world space Z
#             ('m_world_forward_dir_x', ctypes.c_int16), # World space forward X direction (normalised)
#             ('m_world_forward_dir_y', ctypes.c_int16), # World space forward Y direction (normalised)
#             ('m_world_forward_dir_z', ctypes.c_int16), # World space forward Z direction (normalised)
#             ('m_world_right_dir_x', ctypes.c_int16), # World space right X direction (normalised)
#             ('m_world_right_dir_y', ctypes.c_int16), # World space right Y direction (normalised)
#             ('m_world_right_dir_z', ctypes.c_int16), # World space right Z direction (normalised)
#             ('m_g_force_lateral', ctypes.c_float), # Lateral G-Force component
#             ('m_g_force_longitudinal', ctypes.c_float), # Longitudinal G-Force component
#             ('m_g_force_vertical', ctypes.c_float), # Vertical G-Force component
#             ('m_yaw', ctypes.c_float), # Yaw angle in radians
#             ('m_pitch', ctypes.c_float), # Pitch angle in radians
#             ('m_roll', ctypes.c_float), # Roll angle in radians
#         ]
#     )
#     def test_CarMotionData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = CarMotionData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_PacketMotionData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_header', PacketHeader), # Header
#             ('m_car_motion_data', CarMotionData * 22), # Data for all cars on track
#             # Extra player car ONLY data
#             ('m_suspension_position', ctypes.c_float * 4), # Note: All wheel arrays have the following order:
#             ('m_suspension_velocity', ctypes.c_float * 4), # RL, RR, FL, FR
#             ('m_suspension_acceleration', ctypes.c_float * 4), # RL, RR, FL, FR
#             ('m_wheel_speed', ctypes.c_float * 4), # Speed of each wheel
#             ('m_wheel_slip', ctypes.c_float * 4), # Slip ratio for each wheel
#             ('m_local_velocity_x', ctypes.c_float), # Velocity in local space
#             ('m_local_velocity_y', ctypes.c_float), # Velocity in local space
#             ('m_local_velocity_z', ctypes.c_float), # Velocity in local space
#             ('m_angular_velocity_x', ctypes.c_float), # Angular velocity x-component
#             ('m_angular_velocity_y', ctypes.c_float), # Angular velocity y-component
#             ('m_angular_velocity_z', ctypes.c_float), # Angular velocity z-component
#             ('m_angular_acceleration_x', ctypes.c_float), # Angular velocity x-component
#             ('m_angular_acceleration_y', ctypes.c_float), # Angular velocity y-component
#             ('m_angular_acceleration_z', ctypes.c_float), # Angular velocity z-component
#             ('m_front_wheels_angle', ctypes.c_float), # Current front wheels angle in radians
#         ]
#     )
#     def test_PacketMotionData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = PacketMotionData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_MarshalZone(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_zone_start', ctypes.c_float), # Fraction (0..1) of way through the lap the marshal zone starts
#             ('m_zone_flag', ctypes.c_int8), # -1 = invalid/unknown, 0 = none, 1 = green, 2 = blue, 3 = yellow, 4 = red
#         ]
#     )
#     def test_MarshalZone__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = MarshalZone()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_WeatherForecastSample(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_session_type', ctypes.c_uint8), # 0 = unknown, 1 = P1, 2 = P2, 3 = P3, 4 = Short P, 5 = Q1
#             # 6 = Q2, 7 = Q3, 8 = Short Q, 9 = OSQ, 10 = R, 11 = R2
#             # 12 = Time Trial
#             ('m_time_offset', ctypes.c_uint8), # Time in minutes the forecast is for
#             ('m_weather', ctypes.c_uint8), # Weather - 0 = clear, 1 = light cloud, 2 = overcast
#             # 3 = light rain, 4 = heavy rain, 5 = storm
#             ('m_track_temperature', ctypes.c_int8), # Track temp. in degrees Celsius
#             ('m_track_temperature_change', ctypes.c_int8), # Track temp. change – 0 = up, 1 = down, 2 = no change
#             ('m_air_temperature', ctypes.c_int8), # Air temp. in degrees celsius
#             ('m_air_temperature_change', ctypes.c_int8), # Air temp. change – 0 = up, 1 = down, 2 = no change
#             ('m_rain_percentage', ctypes.c_uint8), # Rain percentage (0-100)
#         ]
#     )
#     def test_WeatherForecastSample__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = WeatherForecastSample()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_PacketSessionData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_header', PacketHeader), # Header
#             ('m_weather', ctypes.c_uint8), # Weather - 0 = clear, 1 = light cloud, 2 = overcast
#             # 3 = light rain, 4 = heavy rain, 5 = storm
#             ('m_track_temperature', ctypes.c_int8), # Track temp. in degrees celsius
#             ('m_air_temperature', ctypes.c_int8), # Air temp. in degrees celsius
#             ('m_total_laps', ctypes.c_uint8), # Total number of laps in this race
#             ('m_track_length', ctypes.c_uint16), # Track length in metres
#             ('m_session_type', ctypes.c_uint8), # 0 = unknown, 1 = P1, 2 = P2, 3 = P3, 4 = Short P
#             # 5 = Q1, 6 = Q2, 7 = Q3, 8 = Short Q, 9 = OSQ
#             # 10 = R, 11 = R2, 12 = R3, 13 = Time Trial
#             ('m_track_id', ctypes.c_int8), # -1 for unknown, 0-21 for tracks, see appendix
#             ('m_formula', ctypes.c_uint8), # Formula, 0 = F1 Modern, 1 = F1 Classic, 2 = F2,
#             # 3 = F1 Generic
#             ('m_session_time_left', ctypes.c_uint16), # Time left in session in seconds
#             ('m_session_duration', ctypes.c_uint16), # Session duration in seconds
#             ('m_pit_speed_limit', ctypes.c_uint8), # Pit speed limit in kilometres per hour
#             ('m_game_paused', ctypes.c_uint8), # Whether the game is paused
#             ('m_is_spectating', ctypes.c_uint8), # Whether the player is spectating
#             ('m_spectator_car_index', ctypes.c_uint8), # Index of the car being spectated
#             ('m_sli_pro_native_support', ctypes.c_uint8), # SLI Pro support, 0 = inactive, 1 = active
#             ('m_num_marshal_zones', ctypes.c_uint8), # Number of marshal zones to follow
#             ('m_marshal_zones', MarshalZone * 21), # List of marshal zones – max 21
#             ('m_safety_car_status', ctypes.c_uint8), # 0 = no safety car, 1 = full
#             # 2 = virtual, 3 = formation lap
#             ('m_network_game', ctypes.c_uint8), # 0 = offline, 1 = online
#             ('m_num_weather_forecast_samples', ctypes.c_uint8), # Number of weather samples to follow
#             ('m_weather_forecast_samples', WeatherForecastSample * 56), # Array of weather forecast samples
#             ('m_forecast_accuracy', ctypes.c_uint8), # 0 = Perfect, 1 = Approximate
#             ('m_ai_difficulty', ctypes.c_uint8), # AI Difficulty rating – 0-110
#             ('m_season_link_identifier', ctypes.c_uint32), # Identifier for season - persists across saves
#             ('m_weekend_link_identifier', ctypes.c_uint32), # Identifier for weekend - persists across saves
#             ('m_session_link_identifier', ctypes.c_uint32), # Identifier for session - persists across saves
#             ('m_pit_stop_window_ideal_lap', ctypes.c_uint8), # Ideal lap to pit on for current strategy (player)
#             ('m_pit_stop_window_latest_lap', ctypes.c_uint8), # Latest lap to pit on for current strategy (player)
#             ('m_pit_stop_rejoin_position', ctypes.c_uint8), # Predicted position to rejoin at (player)
#             ('m_steering_assist', ctypes.c_uint8), # 0 = off, 1 = on
#             ('m_braking_assist', ctypes.c_uint8), # 0 = off, 1 = low, 2 = medium, 3 = high
#             ('m_gearbox_assist', ctypes.c_uint8), # 1 = manual, 2 = manual & suggested gear, 3 = auto
#             ('m_pit_assist', ctypes.c_uint8), # 0 = off, 1 = on
#             ('m_pit_release_assist', ctypes.c_uint8), # 0 = off, 1 = on
#             ('m_ersassist', ctypes.c_uint8), # 0 = off, 1 = on
#             ('m_drsassist', ctypes.c_uint8), # 0 = off, 1 = on
#             ('m_dynamic_racing_line', ctypes.c_uint8), # 0 = off, 1 = corners only, 2 = full
#             ('m_dynamic_racing_line_type', ctypes.c_uint8), # 0 = 2D, 1 = 3D
#         ]
#     )
#     def test_PacketSessionData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = PacketSessionData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_LapData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_last_lap_time_in_ms', ctypes.c_uint32), # Last lap time in milliseconds
#             ('m_current_lap_time_in_ms', ctypes.c_uint32), # Current time around the lap in milliseconds
#             ('m_sector1_time_in_ms', ctypes.c_uint16), # Sector 1 time in milliseconds
#             ('m_sector2_time_in_ms', ctypes.c_uint16), # Sector 2 time in milliseconds
#             ('m_lap_distance', ctypes.c_float), # Distance vehicle is around current lap in metres – could
#             # be negative if line hasn’t been crossed yet
#             ('m_total_distance', ctypes.c_float), # Total distance travelled in session in metres – could
#             # be negative if line hasn’t been crossed yet
#             ('m_safety_car_delta', ctypes.c_float), # Delta in seconds for safety car
#             ('m_car_position', ctypes.c_uint8), # Car race position
#             ('m_current_lap_num', ctypes.c_uint8), # Current lap number
#             ('m_pit_status', ctypes.c_uint8), # 0 = none, 1 = pitting, 2 = in pit area
#             ('m_num_pit_stops', ctypes.c_uint8), # Number of pit stops taken in this race
#             ('m_sector', ctypes.c_uint8), # 0 = sector1, 1 = sector2, 2 = sector3
#             ('m_current_lap_invalid', ctypes.c_uint8), # Current lap invalid - 0 = valid, 1 = invalid
#             ('m_penalties', ctypes.c_uint8), # Accumulated time penalties in seconds to be added
#             ('m_warnings', ctypes.c_uint8), # Accumulated number of warnings issued
#             ('m_num_unserved_drive_through_pens', ctypes.c_uint8), # Num drive through pens left to serve
#             ('m_num_unserved_stop_go_pens', ctypes.c_uint8), # Num stop go pens left to serve
#             ('m_grid_position', ctypes.c_uint8), # Grid position the vehicle started the race in
#             ('m_driver_status', ctypes.c_uint8), # Status of driver - 0 = in garage, 1 = flying lap
#             # 2 = in lap, 3 = out lap, 4 = on track
#             ('m_result_status', ctypes.c_uint8), # Result status - 0 = invalid, 1 = inactive, 2 = active
#             # 3 = finished, 4 = didnotfinish, 5 = disqualified
#             # 6 = not classified, 7 = retired
#             ('m_pit_lane_timer_active', ctypes.c_uint8), # Pit lane timing, 0 = inactive, 1 = active
#             ('m_pit_lane_time_in_lane_in_ms', ctypes.c_uint16), # If active, the current time spent in the pit lane in ms
#             ('m_pit_stop_timer_in_ms', ctypes.c_uint16), # Time of the actual pit stop in ms
#             ('m_pit_stop_should_serve_pen', ctypes.c_uint8), # Whether the car should serve a penalty at this stop
#         ]
#     )
#     def test_LapData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = LapData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_PacketLapData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_header', PacketHeader), # Header
#             ('m_lap_data', LapData * 22), # Lap data for all cars on track
#         ]
#     )
#     def test_PacketLapData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = PacketLapData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_FastestLap(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('vehicle_idx', ctypes.c_uint8), # Vehicle index of car achieving fastest lap
#             ('lap_time', ctypes.c_float), # Lap time is in seconds
#         ]
#     )
#     def test_FastestLap__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = FastestLap()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_Retirement(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('vehicle_idx', ctypes.c_uint8), # Vehicle index of car retiring
#         ]
#     )
#     def test_Retirement__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = Retirement()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_TeamMateInPits(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('vehicle_idx', ctypes.c_uint8), # Vehicle index of team mate
#         ]
#     )
#     def test_TeamMateInPits__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = TeamMateInPits()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_RaceWinner(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('vehicle_idx', ctypes.c_uint8), # Vehicle index of the race winner
#         ]
#     )
#     def test_RaceWinner__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = RaceWinner()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_Penalty(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('penalty_type', ctypes.c_uint8), # Penalty type – see Appendices
#             ('infringement_type', ctypes.c_uint8), # Infringement type – see Appendices
#             ('vehicle_idx', ctypes.c_uint8), # Vehicle index of the car the penalty is applied to
#             ('other_vehicle_idx', ctypes.c_uint8), # Vehicle index of the other car involved
#             ('time', ctypes.c_uint8), # Time gained, or time spent doing action in seconds
#             ('lap_num', ctypes.c_uint8), # Lap the penalty occurred on
#             ('places_gained', ctypes.c_uint8), # Number of places gained by this
#         ]
#     )
#     def test_Penalty__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = Penalty()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_SpeedTrap(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('vehicle_idx', ctypes.c_uint8), # Vehicle index of the vehicle triggering speed trap
#             ('speed', ctypes.c_float), # Top speed achieved in kilometres per hour
#             ('overall_fastest_in_session', ctypes.c_uint8), # Overall fastest speed in session = 1, otherwise 0
#             ('driver_fastest_in_session', ctypes.c_uint8), # Fastest speed for driver in session = 1, otherwise 0
#         ]
#     )
#     def test_SpeedTrap__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = SpeedTrap()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_StartLIghts(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('num_lights', ctypes.c_uint8), # Number of lights showing
#         ]
#     )
#     def test_StartLIghts__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = StartLIghts()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_DriveThroughPenaltyServed(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('vehicle_idx', ctypes.c_uint8), # Vehicle index of the vehicle serving drive through
#         ]
#     )
#     def test_DriveThroughPenaltyServed__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = DriveThroughPenaltyServed()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_StopGoPenaltyServed(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('vehicle_idx', ctypes.c_uint8), # Vehicle index of the vehicle serving stop go
#         ]
#     )
#     def test_StopGoPenaltyServed__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = StopGoPenaltyServed()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_Flashback(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('flashback_frame_identifier', ctypes.c_uint32), # Frame identifier flashed back to
#             ('flashback_session_time', ctypes.c_float), # Session time flashed back to
#         ]
#     )
#     def test_Flashback__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = Flashback()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_Buttons(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_button_status', ctypes.c_uint32), # Bit flags specifying which buttons are being pressed
#             # currently - see appendices
#         ]
#     )
#     def test_Buttons__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = Buttons()

#         assert (type_name, type_class) in packet_type._fields_

# # coding: utf-8

# import ctypes

# def to_json(*args, **kwargs):
#     kwargs.setdefault('indent', 2)

#     kwargs['sort_keys'] = True
#     kwargs['ensure_ascii'] = False
#     kwargs['separators'] = (',', ': ')

#     return json.dumps(*args, **kwargs)


# class PacketMixin(object):
#     """A base set of helper methods for ctypes based packets

#     """

#     def get_value(self, field):
#         """Returns the field's value and formats the types value

#         """
#         return self._format_type(getattr(self, field))

#     def pack(self):
#         """Packs the current data structure into a compressed binary

#         Returns:
#             (bytes):
#                 - The packed binary

#         """
#         return bytes(self)

#     @classmethod
#     def size(cls):
#         return ctypes.sizeof(cls)

#     @classmethod
#     def unpack(cls, buffer):
#         """Attempts to unpack the binary structure into a python structure

#         Args:
#             buffer (bytes):
#                 - The encoded buffer to decode

#         """
#         return cls.from_buffer_copy(buffer)

#     def to_dict(self):
#         """Returns a ``dict`` with key-values derived from _fields_

#         """
#         return {k: self.get_value(k) for k, _ in self._fields_}

#     def to_json(self):
#         """Returns a ``str`` of sorted JSON derived from _fields_

#         """
#         return to_json(self.to_dict())

#     def _format_type(self, value):
#         """A type helper to format values

#         """
#         class_name = type(value).__name__

#         if class_name == 'float':
#             return round(value, 3)

#         if class_name == 'bytes':
#             return value.decode()

#         if isinstance(value, ctypes.Array):
#             return self._format_array_type(value)

#         if hasattr(value, 'to_dict'):
#             return value.to_dict()

#         return value

#     def _format_array_type(self, value):
#         results = []

#         for item in value:
#             if isinstance(item, Packet):
#                 results.append(item.to_dict())
#             else:
#                 results.append(item)

#         return results


# class Packet(ctypes.LittleEndianStructure, PacketMixin):
#     """The base packet class for API version 2021

#     """
#     _pack_ = 1

#     def __repr__(self):
#         return self.to_json()


# class EventDataDetails(ctypes.Union, PacketMixin):
#     _fields_ = [
#         ('fastest_lap', FastestLap),
#         ('retirement', Retirement),
#         ('team_mate_in_pits', TeamMateInPits),
#         ('race_winner', RaceWinner),
#         ('penalty', Penalty),
#         ('speed_trap', SpeedTrap),
#         ('start_lights', StartLIghts),
#         ('drive_through_penalty_served', DriveThroughPenaltyServed),
#         ('stop_go_penalty_served', StopGoPenaltyServed),
#         ('flashback', Flashback),
#         ('buttons', Buttons),
#     ]

# class Test_PacketEventData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_header', PacketHeader), # Header
#             ('m_event_string_code', ctypes.c_uint8 * 4), # Event string code, see below
#             ('m_event_details', EventDataDetails), # Event details - should be interpreted differently
#             # for each type
#         ]
#     )
#     def test_PacketEventData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = PacketEventData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_ParticipantData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_ai_controlled', ctypes.c_uint8), # Whether the vehicle is AI (1) or Human (0) controlled
#             ('m_driver_id', ctypes.c_uint8), # Driver id - see appendix, 255 if network human
#             ('m_network_id', ctypes.c_uint8), # Network id – unique identifier for network players
#             ('m_team_id', ctypes.c_uint8), # Team id - see appendix
#             ('m_my_team', ctypes.c_uint8), # My team flag – 1 = My Team, 0 = otherwise
#             ('m_race_number', ctypes.c_uint8), # Race number of the car
#             ('m_nationality', ctypes.c_uint8), # Nationality of the driver
#             ('m_name', ctypes.c_char * 48), # Name of participant in UTF-8 format – null terminated
#             # Will be truncated with … (U+2026) if too long
#             ('m_your_telemetry', ctypes.c_uint8), # The player's UDP setting, 0 = restricted, 1 = public
#         ]
#     )
#     def test_ParticipantData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = ParticipantData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_PacketParticipantsData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_header', PacketHeader), # Header
#             ('m_num_active_cars', ctypes.c_uint8), # Number of active cars in the data – should match number of
#             # cars on HUD
#             ('m_participants', ParticipantData * 22),
#         ]
#     )
#     def test_PacketParticipantsData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = PacketParticipantsData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_CarSetupData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_front_wing', ctypes.c_uint8), # Front wing aero
#             ('m_rear_wing', ctypes.c_uint8), # Rear wing aero
#             ('m_on_throttle', ctypes.c_uint8), # Differential adjustment on throttle (percentage)
#             ('m_off_throttle', ctypes.c_uint8), # Differential adjustment off throttle (percentage)
#             ('m_front_camber', ctypes.c_float), # Front camber angle (suspension geometry)
#             ('m_rear_camber', ctypes.c_float), # Rear camber angle (suspension geometry)
#             ('m_front_toe', ctypes.c_float), # Front toe angle (suspension geometry)
#             ('m_rear_toe', ctypes.c_float), # Rear toe angle (suspension geometry)
#             ('m_front_suspension', ctypes.c_uint8), # Front suspension
#             ('m_rear_suspension', ctypes.c_uint8), # Rear suspension
#             ('m_front_anti_roll_bar', ctypes.c_uint8), # Front anti-roll bar
#             ('m_rear_anti_roll_bar', ctypes.c_uint8), # Front anti-roll bar
#             ('m_front_suspension_height', ctypes.c_uint8), # Front ride height
#             ('m_rear_suspension_height', ctypes.c_uint8), # Rear ride height
#             ('m_brake_pressure', ctypes.c_uint8), # Brake pressure (percentage)
#             ('m_brake_bias', ctypes.c_uint8), # Brake bias (percentage)
#             ('m_rear_left_tyre_pressure', ctypes.c_float), # Rear left tyre pressure (PSI)
#             ('m_rear_right_tyre_pressure', ctypes.c_float), # Rear right tyre pressure (PSI)
#             ('m_front_left_tyre_pressure', ctypes.c_float), # Front left tyre pressure (PSI)
#             ('m_front_right_tyre_pressure', ctypes.c_float), # Front right tyre pressure (PSI)
#             ('m_ballast', ctypes.c_uint8), # Ballast
#             ('m_fuel_load', ctypes.c_float), # Fuel load
#         ]
#     )
#     def test_CarSetupData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = CarSetupData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_PacketCarSetupData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_header', PacketHeader), # Header
#             ('m_car_setups', CarSetupData * 22),
#         ]
#     )
#     def test_PacketCarSetupData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = PacketCarSetupData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_CarTelemetryData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_speed', ctypes.c_uint16), # Speed of car in kilometres per hour
#             ('m_throttle', ctypes.c_float), # Amount of throttle applied (0.0 to 1.0)
#             ('m_steer', ctypes.c_float), # Steering (-1.0 (full lock left) to 1.0 (full lock right))
#             ('m_brake', ctypes.c_float), # Amount of brake applied (0.0 to 1.0)
#             ('m_clutch', ctypes.c_uint8), # Amount of clutch applied (0 to 100)
#             ('m_gear', ctypes.c_int8), # Gear selected (1-8, N=0, R=-1)
#             ('m_engine_rpm', ctypes.c_uint16), # Engine RPM
#             ('m_drs', ctypes.c_uint8), # 0 = off, 1 = on
#             ('m_rev_lights_percent', ctypes.c_uint8), # Rev lights indicator (percentage)
#             ('m_rev_lights_bit_value', ctypes.c_uint16), # Rev lights (bit 0 = leftmost LED, bit 14 = rightmost LED)
#             ('m_brakes_temperature', ctypes.c_uint16 * 4), # Brakes temperature (celsius)
#             ('m_tyres_surface_temperature', ctypes.c_uint8 * 4), # Tyres surface temperature (celsius)
#             ('m_tyres_inner_temperature', ctypes.c_uint8 * 4), # Tyres inner temperature (celsius)
#             ('m_engine_temperature', ctypes.c_uint16), # Engine temperature (celsius)
#             ('m_tyres_pressure', ctypes.c_float * 4), # Tyres pressure (PSI)
#             ('m_surface_type', ctypes.c_uint8 * 4), # Driving surface, see appendices
#         ]
#     )
#     def test_CarTelemetryData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = CarTelemetryData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_PacketCarTelemetryData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_header', PacketHeader), # Header
#             ('m_car_telemetry_data', CarTelemetryData * 22),
#             ('m_mfd_panel_index', ctypes.c_uint8), # Index of MFD panel open - 255 = MFD closed
#             # Single player, race – 0 = Car setup, 1 = Pits
#             # 2 = Damage, 3 =  Engine, 4 = Temperatures
#             # May vary depending on game mode
#             ('m_mfd_panel_index_secondary_player', ctypes.c_uint8), # See above
#             ('m_suggested_gear', ctypes.c_int8), # Suggested gear for the player (1-8)
#             # 0 if no gear suggested
#         ]
#     )
#     def test_PacketCarTelemetryData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = PacketCarTelemetryData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_CarStatusData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_traction_control', ctypes.c_uint8), # Traction control - 0 = off, 1 = medium, 2 = full
#             ('m_anti_lock_brakes', ctypes.c_uint8), # 0 (off) - 1 (on)
#             ('m_fuel_mix', ctypes.c_uint8), # Fuel mix - 0 = lean, 1 = standard, 2 = rich, 3 = max
#             ('m_front_brake_bias', ctypes.c_uint8), # Front brake bias (percentage)
#             ('m_pit_limiter_status', ctypes.c_uint8), # Pit limiter status - 0 = off, 1 = on
#             ('m_fuel_in_tank', ctypes.c_float), # Current fuel mass
#             ('m_fuel_capacity', ctypes.c_float), # Fuel capacity
#             ('m_fuel_remaining_laps', ctypes.c_float), # Fuel remaining in terms of laps (value on MFD)
#             ('m_max_rpm', ctypes.c_uint16), # Cars max RPM, point of rev limiter
#             ('m_idle_rpm', ctypes.c_uint16), # Cars idle RPM
#             ('m_max_gears', ctypes.c_uint8), # Maximum number of gears
#             ('m_drs_allowed', ctypes.c_uint8), # 0 = not allowed, 1 = allowed
#             ('m_drs_activation_distance', ctypes.c_uint16), # 0 = DRS not available, non-zero - DRS will be available
#             # in [X] metres
#             ('m_actual_tyre_compound', ctypes.c_uint8), # F1 Modern - 16 = C5, 17 = C4, 18 = C3, 19 = C2, 20 = C1
#             # 7 = inter, 8 = wet
#             # F1 Classic - 9 = dry, 10 = wet
#             # F2 – 11 = super soft, 12 = soft, 13 = medium, 14 = hard
#             # 15 = wet
#             ('m_visual_tyre_compound', ctypes.c_uint8), # F1 visual (can be different from actual compound)
#             # 16 = soft, 17 = medium, 18 = hard, 7 = inter, 8 = wet
#             # F1 Classic – same as above
#             # F2 ‘19, 15 = wet, 19 – super soft, 20 = soft
#             # 21 = medium , 22 = hard
#             ('m_tyres_age_laps', ctypes.c_uint8), # Age in laps of the current set of tyres
#             ('m_vehicle_fia_flags', ctypes.c_int8), # -1 = invalid/unknown, 0 = none, 1 = green
#             # 2 = blue, 3 = yellow, 4 = red
#             ('m_ers_store_energy', ctypes.c_float), # ERS energy store in Joules
#             ('m_ers_deploy_mode', ctypes.c_uint8), # ERS deployment mode, 0 = none, 1 = medium
#             # 2 = hotlap, 3 = overtake
#             ('m_ers_harvested_this_lap_mguk', ctypes.c_float), # ERS energy harvested this lap by MGU-K
#             ('m_ers_harvested_this_lap_mguh', ctypes.c_float), # ERS energy harvested this lap by MGU-H
#             ('m_ers_deployed_this_lap', ctypes.c_float), # ERS energy deployed this lap
#             ('m_network_paused', ctypes.c_uint8), # Whether the car is paused in a network game
#         ]
#     )
#     def test_CarStatusData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = CarStatusData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_PacketCarStatusData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_header', PacketHeader), # Header
#             ('m_car_status_data', CarStatusData * 22),
#         ]
#     )
#     def test_PacketCarStatusData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = PacketCarStatusData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_FinalClassificationData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_position', ctypes.c_uint8), # Finishing position
#             ('m_num_laps', ctypes.c_uint8), # Number of laps completed
#             ('m_grid_position', ctypes.c_uint8), # Grid position of the car
#             ('m_points', ctypes.c_uint8), # Number of points scored
#             ('m_num_pit_stops', ctypes.c_uint8), # Number of pit stops made
#             ('m_result_status', ctypes.c_uint8), # Result status - 0 = invalid, 1 = inactive, 2 = active
#             # 3 = finished, 4 = didnotfinish, 5 = disqualified
#             # 6 = not classified, 7 = retired
#             ('m_best_lap_time_in_ms', ctypes.c_uint32), # Best lap time of the session in milliseconds
#             ('m_total_race_time', ctypes.c_double), # Total race time in seconds without penalties
#             ('m_penalties_time', ctypes.c_uint8), # Total penalties accumulated in seconds
#             ('m_num_penalties', ctypes.c_uint8), # Number of penalties applied to this driver
#             ('m_num_tyre_stints', ctypes.c_uint8), # Number of tyres stints up to maximum
#             ('m_tyre_stints_actual', ctypes.c_uint8 * 8), # Actual tyres used by this driver
#             ('m_tyre_stints_visual', ctypes.c_uint8 * 8), # Visual tyres used by this driver
#         ]
#     )
#     def test_FinalClassificationData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = FinalClassificationData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_PacketFinalClassificationData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_header', PacketHeader), # Header
#             ('m_num_cars', ctypes.c_uint8), # Number of cars in the final classification
#             ('m_classification_data', FinalClassificationData * 22),
#         ]
#     )
#     def test_PacketFinalClassificationData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = PacketFinalClassificationData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_LobbyInfoData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_ai_controlled', ctypes.c_uint8), # Whether the vehicle is AI (1) or Human (0) controlled
#             ('m_team_id', ctypes.c_uint8), # Team id - see appendix (255 if no team currently selected)
#             ('m_nationality', ctypes.c_uint8), # Nationality of the driver
#             ('m_name', ctypes.c_char * 48), # Name of participant in UTF-8 format – null terminated
#             # Will be truncated with ... (U+2026) if too long
#             ('m_car_number', ctypes.c_uint8), # Car number of the player
#             ('m_ready_status', ctypes.c_uint8), # 0 = not ready, 1 = ready, 2 = spectating
#         ]
#     )
#     def test_LobbyInfoData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = LobbyInfoData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_PacketLobbyInfoData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_header', PacketHeader), # Header
#             # Packet specific data
#             ('m_num_players', ctypes.c_uint8), # Number of players in the lobby data
#             ('m_lobby_players', LobbyInfoData * 22),
#         ]
#     )
#     def test_PacketLobbyInfoData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = PacketLobbyInfoData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_CarDamageData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_tyres_wear', ctypes.c_float * 4), # Tyre wear (percentage)
#             ('m_tyres_damage', ctypes.c_uint8 * 4), # Tyre damage (percentage)
#             ('m_brakes_damage', ctypes.c_uint8 * 4), # Brakes damage (percentage)
#             ('m_front_left_wing_damage', ctypes.c_uint8), # Front left wing damage (percentage)
#             ('m_front_right_wing_damage', ctypes.c_uint8), # Front right wing damage (percentage)
#             ('m_rear_wing_damage', ctypes.c_uint8), # Rear wing damage (percentage)
#             ('m_floor_damage', ctypes.c_uint8), # Floor damage (percentage)
#             ('m_diffuser_damage', ctypes.c_uint8), # Diffuser damage (percentage)
#             ('m_sidepod_damage', ctypes.c_uint8), # Sidepod damage (percentage)
#             ('m_drs_fault', ctypes.c_uint8), # Indicator for DRS fault, 0 = OK, 1 = fault
#             ('m_gear_box_damage', ctypes.c_uint8), # Gear box damage (percentage)
#             ('m_engine_damage', ctypes.c_uint8), # Engine damage (percentage)
#             ('m_engine_mguhwear', ctypes.c_uint8), # Engine wear MGU-H (percentage)
#             ('m_engine_eswear', ctypes.c_uint8), # Engine wear ES (percentage)
#             ('m_engine_cewear', ctypes.c_uint8), # Engine wear CE (percentage)
#             ('m_engine_icewear', ctypes.c_uint8), # Engine wear ICE (percentage)
#             ('m_engine_mgukwear', ctypes.c_uint8), # Engine wear MGU-K (percentage)
#             ('m_engine_tcwear', ctypes.c_uint8), # Engine wear TC (percentage)
#         ]
#     )
#     def test_CarDamageData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = CarDamageData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_PacketCarDamageData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_header', PacketHeader), # Header
#             ('m_car_damage_data', CarDamageData * 22),
#         ]
#     )
#     def test_PacketCarDamageData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = PacketCarDamageData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_LapHistoryData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_lap_time_in_ms', ctypes.c_uint32), # Lap time in milliseconds
#             ('m_sector1_time_in_ms', ctypes.c_uint16), # Sector 1 time in milliseconds
#             ('m_sector2_time_in_ms', ctypes.c_uint16), # Sector 2 time in milliseconds
#             ('m_sector3_time_in_ms', ctypes.c_uint16), # Sector 3 time in milliseconds
#             ('m_lap_valid_bit_flags', ctypes.c_uint8), # 0x01 bit set-lap valid,      0x02 bit set-sector 1 valid
#             # 0x04 bit set-sector 2 valid, 0x08 bit set-sector 3 valid
#         ]
#     )
#     def test_LapHistoryData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = LapHistoryData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_TyreStintHistoryData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_end_lap', ctypes.c_uint8), # Lap the tyre usage ends on (255 of current tyre)
#             ('m_tyre_actual_compound', ctypes.c_uint8), # Actual tyres used by this driver
#             ('m_tyre_visual_compound', ctypes.c_uint8), # Visual tyres used by this driver
#         ]
#     )
#     def test_TyreStintHistoryData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = TyreStintHistoryData()

#         assert (type_name, type_class) in packet_type._fields_


# class Test_PacketSessionHistoryData(object):
#     @pytest.mark.parametrize(
#         'type_name, type_class',
#         [
#             ('m_header', PacketHeader), # Header
#             ('m_car_idx', ctypes.c_uint8), # Index of the car this lap data relates to
#             ('m_num_laps', ctypes.c_uint8), # Num laps in the data (including current partial lap)
#             ('m_num_tyre_stints', ctypes.c_uint8), # Number of tyre stints in the data
#             ('m_best_lap_time_lap_num', ctypes.c_uint8), # Lap the best lap time was achieved on
#             ('m_best_sector1_lap_num', ctypes.c_uint8), # Lap the best Sector 1 time was achieved on
#             ('m_best_sector2_lap_num', ctypes.c_uint8), # Lap the best Sector 2 time was achieved on
#             ('m_best_sector3_lap_num', ctypes.c_uint8), # Lap the best Sector 3 time was achieved on
#             ('m_lap_history_data', LapHistoryData * 100), # 100 laps of data max
#             ('m_tyre_stints_history_data', TyreStintHistoryData * 8),
#         ]
#     )
#     def test_PacketSessionHistoryData__field__types(self, type_name, type_class):  # noqa: E501
#         packet_type = PacketSessionHistoryData()

#         assert (type_name, type_class) in packet_type._fields_

### Rest of UDP

### 2 - Lap Data
Packet Size: 970

In [19]:
# ===========================================================   PACKET ID 2: LAP DATA  ============================================================
class LapData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_last_lap_time_in_ms', ctypes.c_uint32), # Last lap time in milliseconds
        ('m_current_lap_time_in_ms', ctypes.c_uint32), # Current time around the lap in milliseconds
        ('m_sector1_time_in_ms', ctypes.c_uint16), # Sector 1 time in milliseconds
        ('m_sector2_time_in_ms', ctypes.c_uint16), # Sector 2 time in milliseconds
        ('m_lap_distance', ctypes.c_float), # Distance vehicle is around current lap in metres – could        # be negative if line hasn’t been crossed yet
        ('m_total_distance', ctypes.c_float), # Total distance travelled in session in metres – could        # be negative if line hasn’t been crossed yet
        ('m_safety_car_delta', ctypes.c_float), # Delta in seconds for safety car
        ('m_car_position', ctypes.c_uint8), # Car race position
        ('m_current_lap_num', ctypes.c_uint8), # Current lap number
        ('m_pit_status', ctypes.c_uint8), # 0 = none, 1 = pitting, 2 = in pit area
        ('m_num_pit_stops', ctypes.c_uint8), # Number of pit stops taken in this race
        ('m_sector', ctypes.c_uint8), # 0 = sector1, 1 = sector2, 2 = sector3
        ('m_current_lap_invalid', ctypes.c_uint8), # Current lap invalid - 0 = valid, 1 = invalid
        ('m_penalties', ctypes.c_uint8), # Accumulated time penalties in seconds to be added
        ('m_warnings', ctypes.c_uint8), # Accumulated number of warnings issued
        ('m_num_unserved_drive_through_pens', ctypes.c_uint8), # Num drive through pens left to serve
        ('m_num_unserved_stop_go_pens', ctypes.c_uint8), # Num stop go pens left to serve
        ('m_grid_position', ctypes.c_uint8), # Grid position the vehicle started the race in
        ('m_driver_status', ctypes.c_uint8), # Status of driver - 0 = in garage, 1 = flying lap        # 2 = in lap, 3 = out lap, 4 = on track
        ('m_result_status', ctypes.c_uint8), # Result status - 0 = invalid, 1 = inactive, 2 = active        # 3 = finished, 4 = didnotfinish, 5 = disqualified        # 6 = not classified, 7 = retired
        ('m_pit_lane_timer_active', ctypes.c_uint8), # Pit lane timing, 0 = inactive, 1 = active
        ('m_pit_lane_time_in_lane_in_ms', ctypes.c_uint16), # If active, the current time spent in the pit lane in ms
        ('m_pit_stop_timer_in_ms', ctypes.c_uint16), # Time of the actual pit stop in ms
        ('m_pit_stop_should_serve_pen', ctypes.c_uint8), # Whether the car should serve a penalty at this stop
    ]

class PacketLapData(ctypes.LittleEndianStructure):
    _fields_ = [
        ('m_header', PacketHeader), # Header
        ('m_lap_data', LapData * 22), # Lap data for all cars on track
    ]


In [10]:
# # READ DATA FROM data.txt
# with open("data.txt", "rb") as f:
#     data = f.readlines()

#     f.close()

# l = []


# for i in range(len(data)):
#     try:
#         # print(i)

#         data[i] = data[i].replace(b"b'", b"")
#         data[i] = data[i].replace(b'"b""', b"")
#         data[i] = data[i].replace(b"\r", b"")
#         data[i] = data[i].replace(b"'\n", b"")
#         data[i] = data[i].replace(b'"\n', b"")

#         escaped_byte_literal = data[i]
#         data[i] = escaped_byte_literal.decode('unicode_escape').encode('raw_unicode_escape')
        
#     except:
#         pass
    
# data = data[1:]
# data[0:10]

with open('motion_data.txt', 'rb') as f:
    data = f.readlines()
    f.close()
    
bin = []


for line in data:
    bin.append(line.decode())
    
data = []

for line in bin:
    data.append(bytes.fromhex(line))

In [11]:
# DATA SAMPLES FROM WIRESHARK
# Packet ID 2, len 970
# d = b"\xe5\x07\x01\x12\x01\x02\x08\xcd\x7f\x68\xcf\x97\xbb\x60\x0d\xdc\x6c\x42\xe4\x06\x00\x00\x00\xff\x00\x00\x00\x00\x1e\x5e\x00\x00\x00\x00\x00\x00\x9d\xea\xae\x44\x9d\xea\xae\x44\x00\x00\x00\x80\x01\x01\x00\x00\x00\x01\x00\x00\x00\x00\x01\x01\x02\x00\x00\x00\x00\x00\x00\xe5\x70\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x74\xe3\x6b\x43\x74\xe3\x6b\x43\x00\x00\x00\x80\x01\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x92\xd1\x63\x43\x92\xd1\x63\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x40\xe6\x5b\x43\x40\xe6\x5b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xb6\x53\x43\xcd\xb6\x53\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfd\x4b\x43\x00\xfd\x4b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc2\xb2\x43\x43\xc2\xb2\x43\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x52\xcb\x3b\x43\x52\xcb\x3b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x6c\xb2\x33\x43\x6c\xb2\x33\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11\xdc\x2b\x43\x11\xdc\x2b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb2\xc6\x23\x43\xb2\xc6\x23\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb9\xbc\x1b\x43\xb9\xbc\x1b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3d\xbe\x13\x43\x3d\xbe\x13\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x62\x03\x0c\x43\x62\x03\x0c\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"
# data = [b"\xe5\x07\x01\x12\x01\x02\x08\xcd\x7f\x68\xcf\x97\xbb\x60\xd9\xf0\x97\x42\xda\x08\x00\x00\x00\xff\x00\x00\x00\x00\x92\x9f\x00\x00\x00\x00\x00\x00\xf6\x1a\xc5\x44\xf6\x1a\xc5\x44\x00\x00\x00\x80\x01\x01\x00\x00\x00\x01\x00\x00\x00\x00\x01\x01\x02\x00\x00\x00\x00\x00\x00\xe5\x70\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x74\xe3\x6b\x43\x74\xe3\x6b\x43\x00\x00\x00\x80\x01\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x92\xd1\x63\x43\x92\xd1\x63\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x40\xe6\x5b\x43\x40\xe6\x5b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xb6\x53\x43\xcd\xb6\x53\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfd\x4b\x43\x00\xfd\x4b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc2\xb2\x43\x43\xc2\xb2\x43\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x52\xcb\x3b\x43\x52\xcb\x3b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x6c\xb2\x33\x43\x6c\xb2\x33\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11\xdc\x2b\x43\x11\xdc\x2b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb2\xc6\x23\x43\xb2\xc6\x23\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb9\xbc\x1b\x43\xb9\xbc\x1b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3d\xbe\x13\x43\x3d\xbe\x13\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x62\x03\x0c\x43\x62\x03\x0c\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"
# ,b"\xe5\x07\x01\x12\x01\x02\x08\xcd\x7f\x68\xcf\x97\xbb\x60\xec\x01\x98\x42\xdb\x08\x00\x00\x00\xff\x00\x00\x00\x00\xb3\x9f\x00\x00\x00\x00\x00\x00\xf6\x1a\xc5\x44\xf6\x1a\xc5\x44\x00\x00\x00\x80\x01\x01\x00\x00\x00\x01\x00\x00\x00\x00\x01\x01\x02\x00\x00\x00\x00\x00\x00\xe5\x70\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x74\xe3\x6b\x43\x74\xe3\x6b\x43\x00\x00\x00\x80\x01\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x92\xd1\x63\x43\x92\xd1\x63\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x40\xe6\x5b\x43\x40\xe6\x5b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xb6\x53\x43\xcd\xb6\x53\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfd\x4b\x43\x00\xfd\x4b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc2\xb2\x43\x43\xc2\xb2\x43\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x52\xcb\x3b\x43\x52\xcb\x3b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x6c\xb2\x33\x43\x6c\xb2\x33\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11\xdc\x2b\x43\x11\xdc\x2b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb2\xc6\x23\x43\xb2\xc6\x23\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb9\xbc\x1b\x43\xb9\xbc\x1b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3d\xbe\x13\x43\x3d\xbe\x13\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x62\x03\x0c\x43\x62\x03\x0c\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"
# ,b"\xe5\x07\x01\x12\x01\x02\x08\xcd\x7f\x68\xcf\x97\xbb\x60\x01\x13\x98\x42\xdc\x08\x00\x00\x00\xff\x00\x00\x00\x00\xd4\x9f\x00\x00\x00\x00\x00\x00\xf6\x1a\xc5\x44\xf6\x1a\xc5\x44\x00\x00\x00\x80\x01\x01\x00\x00\x00\x01\x00\x00\x00\x00\x01\x01\x02\x00\x00\x00\x00\x00\x00\xe5\x70\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x74\xe3\x6b\x43\x74\xe3\x6b\x43\x00\x00\x00\x80\x01\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x92\xd1\x63\x43\x92\xd1\x63\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x40\xe6\x5b\x43\x40\xe6\x5b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xb6\x53\x43\xcd\xb6\x53\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfd\x4b\x43\x00\xfd\x4b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc2\xb2\x43\x43\xc2\xb2\x43\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x52\xcb\x3b\x43\x52\xcb\x3b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x6c\xb2\x33\x43\x6c\xb2\x33\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11\xdc\x2b\x43\x11\xdc\x2b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb2\xc6\x23\x43\xb2\xc6\x23\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb9\xbc\x1b\x43\xb9\xbc\x1b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3d\xbe\x13\x43\x3d\xbe\x13\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x62\x03\x0c\x43\x62\x03\x0c\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"
# ]

# d = b"\xe5\x07\x01\x12\x01\x02\x08\xcd\x7f\x68\xcf\x97\xbb\x60\x07\x24\x8c\x42\x29\x08\x00\x00\x00\xff\x00\x00\x00\x00\x86\x88\x00\x00\x00\x00\x00\x00\x01\x85\xc4\x44\x01\x85\xc4\x44\x00\x00\x00\x80\x01\x01\x00\x00\x00\x01\x00\x00\x00\x00\x01\x01\x02\x00\x00\x00\x00\x00\x00\xe5\x70\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x74\xe3\x6b\x43\x74\xe3\x6b\x43\x00\x00\x00\x80\x01\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x92\xd1\x63\x43\x92\xd1\x63\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x40\xe6\x5b\x43\x40\xe6\x5b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xb6\x53\x43\xcd\xb6\x53\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfd\x4b\x43\x00\xfd\x4b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc2\xb2\x43\x43\xc2\xb2\x43\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x52\xcb\x3b\x43\x52\xcb\x3b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x6c\xb2\x33\x43\x6c\xb2\x33\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11\xdc\x2b\x43\x11\xdc\x2b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb2\xc6\x23\x43\xb2\xc6\x23\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb9\xbc\x1b\x43\xb9\xbc\x1b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3d\xbe\x13\x43\x3d\xbe\x13\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x62\x03\x0c\x43\x62\x03\x0c\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"
# d = b"\xe5\x07\x01\x12\x01\x02\x08\xcd\x7f\x68\xcf\x97\xbb\x60\x14\x35\x8c\x42\x2a\x08\x00\x00\x00\xff\x00\x00\x00\x00\xa7\x88\x00\x00\x00\x00\x00\x00\x19\x88\xc4\x44\x19\x88\xc4\x44\x00\x00\x00\x80\x01\x01\x00\x00\x00\x01\x00\x00\x00\x00\x01\x01\x02\x00\x00\x00\x00\x00\x00\xe5\x70\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x74\xe3\x6b\x43\x74\xe3\x6b\x43\x00\x00\x00\x80\x01\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x92\xd1\x63\x43\x92\xd1\x63\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x40\xe6\x5b\x43\x40\xe6\x5b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xb6\x53\x43\xcd\xb6\x53\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfd\x4b\x43\x00\xfd\x4b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc2\xb2\x43\x43\xc2\xb2\x43\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x52\xcb\x3b\x43\x52\xcb\x3b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x6c\xb2\x33\x43\x6c\xb2\x33\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11\xdc\x2b\x43\x11\xdc\x2b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb2\xc6\x23\x43\xb2\xc6\x23\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb9\xbc\x1b\x43\xb9\xbc\x1b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3d\xbe\x13\x43\x3d\xbe\x13\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x62\x03\x0c\x43\x62\x03\x0c\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"
# d = b"\xe5\x07\x01\x12\x01\x02\x08\xcd\x7f\x68\xcf\x97\xbb\x60\x37\x57\x8c\x42\x2c\x08\x00\x00\x00\xff\x00\x00\x00\x00\xea\x88\x00\x00\x00\x00\x00\x00\x92\x8e\xc4\x44\x92\x8e\xc4\x44\x00\x00\x00\x80\x01\x01\x00\x00\x00\x01\x00\x00\x00\x00\x01\x01\x02\x00\x00\x00\x00\x00\x00\xe5\x70\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x74\xe3\x6b\x43\x74\xe3\x6b\x43\x00\x00\x00\x80\x01\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x92\xd1\x63\x43\x92\xd1\x63\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x40\xe6\x5b\x43\x40\xe6\x5b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xb6\x53\x43\xcd\xb6\x53\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfd\x4b\x43\x00\xfd\x4b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc2\xb2\x43\x43\xc2\xb2\x43\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x52\xcb\x3b\x43\x52\xcb\x3b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x6c\xb2\x33\x43\x6c\xb2\x33\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11\xdc\x2b\x43\x11\xdc\x2b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb2\xc6\x23\x43\xb2\xc6\x23\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb9\xbc\x1b\x43\xb9\xbc\x1b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3d\xbe\x13\x43\x3d\xbe\x13\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x62\x03\x0c\x43\x62\x03\x0c\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"
# d = b"\xe5\x07\x01\x12\x01\x02\x08\xcd\x7f\x68\xcf\x97\xbb\x60\x6b\x8a\x8c\x42\x2f\x08\x00\x00\x00\xff\x00\x00\x00\x00\x4e\x89\x00\x00\x00\x00\x00\x00\x14\x99\xc4\x44\x14\x99\xc4\x44\x00\x00\x00\x80\x01\x01\x00\x00\x00\x01\x00\x00\x00\x00\x01\x01\x02\x00\x00\x00\x00\x00\x00\xe5\x70\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x74\xe3\x6b\x43\x74\xe3\x6b\x43\x00\x00\x00\x80\x01\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x92\xd1\x63\x43\x92\xd1\x63\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x40\xe6\x5b\x43\x40\xe6\x5b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xb6\x53\x43\xcd\xb6\x53\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfd\x4b\x43\x00\xfd\x4b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc2\xb2\x43\x43\xc2\xb2\x43\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x52\xcb\x3b\x43\x52\xcb\x3b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x6c\xb2\x33\x43\x6c\xb2\x33\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11\xdc\x2b\x43\x11\xdc\x2b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb2\xc6\x23\x43\xb2\xc6\x23\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb9\xbc\x1b\x43\xb9\xbc\x1b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3d\xbe\x13\x43\x3d\xbe\x13\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x62\x03\x0c\x43\x62\x03\x0c\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"
# d = bytes.fromhex('e5070112010008cd7f68cf97bb603bfe6e42f406000000ff6749924351a4d342b051f5c3402a36420ee8503f193117414b7d54020e1a02e6a2fc487d5cd9a3bdf22db13f88de0abcf1e0ae3f40fdbdbc59c7d73c124dbcc3a3a9b642f1d7c3c20000000000000000000000005a0514001e80de7ff0015a050000000000000000000000007b624640805e78bc792878bc0f0ebfc3fb80b6426025b4c2000000000000000000000000b70601002f80cd7ff901b606000000000000000000000000c3b3454080547cbc20a37cbc84a5bcc386adb64299e4a3c2000000000000000000000000e503e0ff1180eb7f0c02e403000000000000000000000000e51c474000f282bc7c4083bc1b67bfc3cd84b64213f893c2000000000000000000000000000704003380c97f0c020007000000000000000000000000068f4540400b83bccc2d83bcd2febcc31aacb642531984c2000000000000000000000000c0030d001080ed7ff301c003000000000000000000000000a42f474080c679bc3dad79bc1fc0bfc3e982b64282f067c2000000000000000000000000340717003580c77fe00134070000000000000000000000002775454000b570bcb16a70bc2d57bdc32ca8b642cc7947c2000000000000000000000000c80504002380d97ff101c8050000000000000000000000003e2b464080c178bcf7ea78bc1619c0c37b83b642baff27c2000000000000000000000000c005c3ff2380da7fee01bf05000000000000000000000000582f4640808b75bc8c2b77bc82b0bdc3baadb64244cb07c2000000000000000000000000bd0602002f80ce7fd601bd06000000000000000000000000b8b0454080d66abcc8196bbc5074c0c38884b64289c4d0c1000000000000000000000000820624002c80d07fe70182060000000000000000000000001ace4540804d74bc1fae73bced0abec35faab64235b98ec1000000000000000000000000790610002b80d17f000279060000000000000000000000008ad24540001280bcff0680bc5accc0c38681b642d44221c10000000000000000000000000505deff1b80e17f05020405000000000000000000000000f98c464080f580bc9e6481bc5b64bec39eacb6429c6008c00000000000000000000000004404ebff1480e97fe901430400000000000000000000000091ed4640806774bcfbe474bc0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000005414a6c0426ad6c022947c40131b18407ee3d4c0c715c241a4f649c125875941180257c3f1c98c447d4d39c41b53e0436e643c42dfb23b42c63b3a4222b639420799373caae7253c334001bac8eb1bba5ba4a93cb04802bddf123a42cd45923d23879abdc971723c401634400aef913fc6ff073f10745c3a')

# #Packet ID 11, len 1155
# d = b"\xe5\x07\x01\x12\x01\x0b\x08\xcd\x7f\x68\xcf\x97\xbb\x60\x0d\xdc\x6c\x42\xe4\x06\x00\x00\x00\xff\x07\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0f\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\x11\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"

# # Packet ID 7, len 1058
# d = b"\xe5\x07\x01\x12\x01\x07\x08\xcd\x7f\x68\xcf\x97\xbb\x60\xf0\xb9\x6c\x42\xe3\x06\x00\x00\x00\xff\x01\x00\x03\x3a\x00\x00\x00\x20\x41\x00\x00\xdc\x42\x60\x1d\xad\x40\xc8\x32\xcc\x10\x09\x01\x00\x00\x11\x10\x00\x00\x00\x24\x74\x4a\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x3c\x00\x00\x00\x20\x41\x00\x00\xdc\x42\x60\x1d\xad\x40\xc8\x32\xcc\x10\x09\x01\x00\x00\x11\x10\x01\x00\x00\x24\x74\x4a\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x3c\x00\x00\x00\x20\x41\x00\x00\xdc\x42\x60\x1d\xad\x40\xc8\x32\xcc\x10\x09\x01\x00\x00\x11\x10\x00\x00\x00\x24\x74\x4a\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x3c\x00\x00\x00\x20\x41\x00\x00\xdc\x42\x60\x1d\xad\x40\xc8\x32\xcc\x10\x09\x01\x00\x00\x11\x10\x00\x00\x00\x24\x74\x4a\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x3c\x00\x00\x00\x20\x41\x00\x00\xdc\x42\x60\x1d\xad\x40\xc8\x32\xcc\x10\x09\x01\x00\x00\x11\x10\x00\x00\x00\x24\x74\x4a\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x3c\x00\x00\x00\x20\x41\x00\x00\xdc\x42\x60\x1d\xad\x40\xc8\x32\xcc\x10\x09\x01\x00\x00\x11\x10\x00\x00\x00\x24\x74\x4a\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x3c\x00\x00\x00\x20\x41\x00\x00\xdc\x42\x60\x1d\xad\x40\xc8\x32\xcc\x10\x09\x01\x00\x00\x11\x10\x00\x00\x00\x24\x74\x4a\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x3c\x00\x00\x00\x20\x41\x00\x00\xdc\x42\x60\x1d\xad\x40\xc8\x32\xcc\x10\x09\x01\x00\x00\x11\x10\x00\x00\x00\x24\x74\x4a\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x3c\x00\x00\x00\x20\x41\x00\x00\xdc\x42\x60\x1d\xad\x40\xc8\x32\xcc\x10\x09\x01\x00\x00\x11\x10\x00\x00\x00\x24\x74\x4a\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x3c\x00\x00\x00\x20\x41\x00\x00\xdc\x42\x60\x1d\xad\x40\xc8\x32\xcc\x10\x09\x01\x00\x00\x11\x10\x00\x00\x00\x24\x74\x4a\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x3c\x00\x00\x00\x20\x41\x00\x00\xdc\x42\x60\x1d\xad\x40\xc8\x32\xcc\x10\x09\x01\x00\x00\x11\x10\x00\x00\x00\x24\x74\x4a\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x3c\x00\x00\x00\x20\x41\x00\x00\xdc\x42\x60\x1d\xad\x40\xc8\x32\xcc\x10\x09\x01\x00\x00\x11\x10\x00\x00\x00\x24\x74\x4a\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x3c\x00\x00\x00\x20\x41\x00\x00\xdc\x42\x60\x1d\xad\x40\xc8\x32\xcc\x10\x09\x01\x00\x00\x11\x10\x00\x00\x00\x24\x74\x4a\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x3c\x00\x00\x00\x20\x41\x00\x00\xdc\x42\x60\x1d\xad\x40\xc8\x32\xcc\x10\x09\x01\x00\x00\x11\x10\x00\x00\x00\x24\x74\x4a\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"

# # Packet ID 6, len 1347
# d = b"\xe5\x07\x01\x12\x01\x06\x08\xcd\x7f\x68\xcf\x97\xbb\x60\xf0\xb9\x6c\x42\xe3\x06\x00\x00\x00\xff\x8c\x00\x00\x00\x80\x3f\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x07\x2b\x00\x04\x00\x40\x24\x00\x24\x00\x24\x00\x24\x00\x64\x64\x64\x64\x64\x64\x64\x64\x5a\x00\x2c\x97\xbc\x41\x2c\x97\xbc\x41\x2a\x3e\xb5\x41\x2a\x3e\xb5\x41\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\xcc\x10\x00\x00\x00\x40\x24\x00\x24\x00\x24\x00\x24\x00\x55\x55\x5a\x5a\x55\x55\x5a\x5a\x5a\x00\x00\x00\xac\x41\x00\x00\xac\x41\x00\x00\xb8\x41\x00\x00\xb8\x41\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\xcc\x10\x00\x00\x00\x40\x24\x00\x24\x00\x24\x00\x24\x00\x55\x55\x5a\x5a\x55\x55\x5a\x5a\x5a\x00\x00\x00\xac\x41\x00\x00\xac\x41\x00\x00\xb8\x41\x00\x00\xb8\x41\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\xcc\x10\x00\x00\x00\x40\x24\x00\x24\x00\x24\x00\x24\x00\x55\x55\x5a\x5a\x55\x55\x5a\x5a\x5a\x00\x00\x00\xac\x41\x00\x00\xac\x41\x00\x00\xb8\x41\x00\x00\xb8\x41\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\xcc\x10\x00\x00\x00\x40\x24\x00\x24\x00\x24\x00\x24\x00\x55\x55\x5a\x5a\x55\x55\x5a\x5a\x5a\x00\x00\x00\xac\x41\x00\x00\xac\x41\x00\x00\xb8\x41\x00\x00\xb8\x41\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\xcc\x10\x00\x00\x00\x40\x24\x00\x24\x00\x24\x00\x24\x00\x55\x55\x5a\x5a\x55\x55\x5a\x5a\x5a\x00\x00\x00\xac\x41\x00\x00\xac\x41\x00\x00\xb8\x41\x00\x00\xb8\x41\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\xcc\x10\x00\x00\x00\x40\x24\x00\x24\x00\x24\x00\x24\x00\x55\x55\x5a\x5a\x55\x55\x5a\x5a\x5a\x00\x00\x00\xac\x41\x00\x00\xac\x41\x00\x00\xb8\x41\x00\x00\xb8\x41\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\xcc\x10\x00\x00\x00\x40\x24\x00\x24\x00\x24\x00\x24\x00\x55\x55\x5a\x5a\x55\x55\x5a\x5a\x5a\x00\x00\x00\xac\x41\x00\x00\xac\x41\x00\x00\xb8\x41\x00\x00\xb8\x41\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\xcc\x10\x00\x00\x00\x40\x24\x00\x24\x00\x24\x00\x24\x00\x55\x55\x5a\x5a\x55\x55\x5a\x5a\x5a\x00\x00\x00\xac\x41\x00\x00\xac\x41\x00\x00\xb8\x41\x00\x00\xb8\x41\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\xcc\x10\x00\x00\x00\x40\x24\x00\x24\x00\x24\x00\x24\x00\x55\x55\x5a\x5a\x55\x55\x5a\x5a\x5a\x00\x00\x00\xac\x41\x00\x00\xac\x41\x00\x00\xb8\x41\x00\x00\xb8\x41\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\xcc\x10\x00\x00\x00\x40\x24\x00\x24\x00\x24\x00\x24\x00\x55\x55\x5a\x5a\x55\x55\x5a\x5a\x5a\x00\x00\x00\xac\x41\x00\x00\xac\x41\x00\x00\xb8\x41\x00\x00\xb8\x41\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\xcc\x10\x00\x00\x00\x40\x24\x00\x24\x00\x24\x00\x24\x00\x55\x55\x5a\x5a\x55\x55\x5a\x5a\x5a\x00\x00\x00\xac\x41\x00\x00\xac\x41\x00\x00\xb8\x41\x00\x00\xb8\x41\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\xcc\x10\x00\x00\x00\x40\x24\x00\x24\x00\x24\x00\x24\x00\x55\x55\x5a\x5a\x55\x55\x5a\x5a\x5a\x00\x00\x00\xac\x41\x00\x00\xac\x41\x00\x00\xb8\x41\x00\x00\xb8\x41\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\xcc\x10\x00\x00\x00\x40\x24\x00\x24\x00\x24\x00\x24\x00\x55\x55\x5a\x5a\x55\x55\x5a\x5a\x5a\x00\x00\x00\xac\x41\x00\x00\xac\x41\x00\x00\xb8\x41\x00\x00\xb8\x41\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\x00"

# # Packet ID 0, len 1464
# d = b"\xe5\x07\x01\x12\x01\x00\x08\xcd\x7f\x68\xcf\x97\xbb\x60\xf0\xb9\x6c\x42\xe3\x06\x00\x00\x00\xff\x4b\x4c\x86\x43\xbc\xbb\xd2\x42\xb3\xb5\xf7\xc3\x29\xf1\x18\x42\x49\x83\x37\x3f\x86\x3a\xf2\x40\x8b\x7d\x6c\x02\xcf\x18\x44\xe7\x73\xfc\x88\x7d\xdc\x2a\x26\x39\x4e\x43\xce\x3f\xc8\x61\x08\xbb\xcd\x28\xb0\x3f\x00\x47\xc4\xbc\x4f\x4d\xe3\x3c\x12\x4d\xbc\xc3\xa3\xa9\xb6\x42\xf1\xd7\xc3\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x5a\x05\x14\x00\x1e\x80\xde\x7f\xf0\x01\x5a\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x7b\x62\x46\x40\x80\x5e\x78\xbc\x79\x28\x78\xbc\x0f\x0e\xbf\xc3\xfb\x80\xb6\x42\x60\x25\xb4\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb7\x06\x01\x00\x2f\x80\xcd\x7f\xf9\x01\xb6\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc3\xb3\x45\x40\x80\x54\x7c\xbc\x20\xa3\x7c\xbc\x84\xa5\xbc\xc3\x86\xad\xb6\x42\x99\xe4\xa3\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe5\x03\xe0\xff\x11\x80\xeb\x7f\x0c\x02\xe4\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe5\x1c\x47\x40\x00\xf2\x82\xbc\x7c\x40\x83\xbc\x1b\x67\xbf\xc3\xcd\x84\xb6\x42\x13\xf8\x93\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x07\x04\x00\x33\x80\xc9\x7f\x0c\x02\x00\x07\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x06\x8f\x45\x40\x40\x0b\x83\xbc\xcc\x2d\x83\xbc\xd2\xfe\xbc\xc3\x1a\xac\xb6\x42\x53\x19\x84\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0\x03\x0d\x00\x10\x80\xed\x7f\xf3\x01\xc0\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa4\x2f\x47\x40\x80\xc6\x79\xbc\x3d\xad\x79\xbc\x1f\xc0\xbf\xc3\xe9\x82\xb6\x42\x82\xf0\x67\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x34\x07\x17\x00\x35\x80\xc7\x7f\xe0\x01\x34\x07\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x27\x75\x45\x40\x00\xb5\x70\xbc\xb1\x6a\x70\xbc\x2d\x57\xbd\xc3\x2c\xa8\xb6\x42\xcc\x79\x47\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc8\x05\x04\x00\x23\x80\xd9\x7f\xf1\x01\xc8\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3e\x2b\x46\x40\x80\xc1\x78\xbc\xf7\xea\x78\xbc\x16\x19\xc0\xc3\x7b\x83\xb6\x42\xba\xff\x27\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0\x05\xc3\xff\x23\x80\xda\x7f\xee\x01\xbf\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x58\x2f\x46\x40\x80\x8b\x75\xbc\x8c\x2b\x77\xbc\x82\xb0\xbd\xc3\xba\xad\xb6\x42\x44\xcb\x07\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xbd\x06\x02\x00\x2f\x80\xce\x7f\xd6\x01\xbd\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb8\xb0\x45\x40\x80\xd6\x6a\xbc\xc8\x19\x6b\xbc\x50\x74\xc0\xc3\x88\x84\xb6\x42\x89\xc4\xd0\xc1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x82\x06\x24\x00\x2c\x80\xd0\x7f\xe7\x01\x82\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x1a\xce\x45\x40\x80\x4d\x74\xbc\x1f\xae\x73\xbc\xed\x0a\xbe\xc3\x5f\xaa\xb6\x42\x35\xb9\x8e\xc1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x79\x06\x10\x00\x2b\x80\xd1\x7f\x00\x02\x79\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x8a\xd2\x45\x40\x00\x12\x80\xbc\xff\x06\x80\xbc\x5a\xcc\xc0\xc3\x86\x81\xb6\x42\xd4\x42\x21\xc1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x05\xde\xff\x1b\x80\xe1\x7f\x05\x02\x04\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf9\x8c\x46\x40\x80\xf5\x80\xbc\x9e\x64\x81\xbc\x5b\x64\xbe\xc3\x9e\xac\xb6\x42\x9c\x60\x08\xc0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x44\x04\xeb\xff\x14\x80\xe9\x7f\xe9\x01\x43\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x91\xed\x46\x40\x80\x67\x74\xbc\xfb\xe4\x74\xbc\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc5\xe5\xf7\xc0\x69\x0c\xf3\xc0\x6b\xda\x9c\x3f\xb1\xa5\xb2\x3f\xf6\x0b\x08\x40\xf3\x00\x19\x40\x3e\xae\x0a\x40\xb2\x4a\x0c\x40\xf4\x37\xa5\x41\xdb\x29\xc2\x41\xd3\x1a\x2c\x42\x78\xaa\x19\x42\x73\x65\x25\x42\x36\xe7\x24\x42\x30\xd0\x1b\x42\x4a\xd0\x1b\x42\x5c\x0e\x79\x3d\x1f\x2a\x6c\x3d\xd3\xa8\x4f\xba\xc4\x4c\x4e\xba\x0c\xa9\x69\xbc\xc0\x81\xae\xbc\xcc\xef\x1b\x42\x09\x03\x2d\xba\x2f\x2b\xca\xb8\xd8\x4e\xab\xb9\xfc\xe2\x17\xbd\x89\xd8\x03\xbc\x28\x42\x87\x3c\x00\x00\x00\x80"
# d = b"\xe5\x07\x01\x12\x01\x00\x08\xcd\x7f\x68\xcf\x97\xbb\x60\xa3\xb9\x76\x42\x2e\x07\x00\x00\x00\xff\xd2\xf0\xbd\x43\x99\xdd\xd6\x42\x97\x3c\xeb\xc3\x12\x91\x28\x42\x8b\x7c\x55\x3f\xec\x9a\x1f\x41\x8d\x7c\xfe\x01\x6c\x1d\xa0\xe2\x36\xfd\x8c\x7c\xeb\x71\x1e\xbd\xbf\x65\xb4\xbe\x7a\x57\xb5\x3e\xec\x69\xab\x3f\x80\x35\xa5\xbc\x83\xb0\xb2\x3c\x12\x4d\xbc\xc3\xa3\xa9\xb6\x42\xf1\xd7\xc3\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x5a\x05\x14\x00\x1e\x80\xde\x7f\xf0\x01\x5a\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x7b\x62\x46\x40\x80\x5e\x78\xbc\x79\x28\x78\xbc\x0f\x0e\xbf\xc3\xfb\x80\xb6\x42\x60\x25\xb4\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb7\x06\x01\x00\x2f\x80\xcd\x7f\xf9\x01\xb6\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc3\xb3\x45\x40\x80\x54\x7c\xbc\x20\xa3\x7c\xbc\x84\xa5\xbc\xc3\x86\xad\xb6\x42\x99\xe4\xa3\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe5\x03\xe0\xff\x11\x80\xeb\x7f\x0c\x02\xe4\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe5\x1c\x47\x40\x00\xf2\x82\xbc\x7c\x40\x83\xbc\x1b\x67\xbf\xc3\xcd\x84\xb6\x42\x13\xf8\x93\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x07\x04\x00\x33\x80\xc9\x7f\x0c\x02\x00\x07\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x06\x8f\x45\x40\x40\x0b\x83\xbc\xcc\x2d\x83\xbc\xd2\xfe\xbc\xc3\x1a\xac\xb6\x42\x53\x19\x84\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0\x03\x0d\x00\x10\x80\xed\x7f\xf3\x01\xc0\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa4\x2f\x47\x40\x80\xc6\x79\xbc\x3d\xad\x79\xbc\x1f\xc0\xbf\xc3\xe9\x82\xb6\x42\x82\xf0\x67\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x34\x07\x17\x00\x35\x80\xc7\x7f\xe0\x01\x34\x07\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x27\x75\x45\x40\x00\xb5\x70\xbc\xb1\x6a\x70\xbc\x2d\x57\xbd\xc3\x2c\xa8\xb6\x42\xcc\x79\x47\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc8\x05\x04\x00\x23\x80\xd9\x7f\xf1\x01\xc8\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3e\x2b\x46\x40\x80\xc1\x78\xbc\xf7\xea\x78\xbc\x16\x19\xc0\xc3\x7b\x83\xb6\x42\xba\xff\x27\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0\x05\xc3\xff\x23\x80\xda\x7f\xee\x01\xbf\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x58\x2f\x46\x40\x80\x8b\x75\xbc\x8c\x2b\x77\xbc\x82\xb0\xbd\xc3\xba\xad\xb6\x42\x44\xcb\x07\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xbd\x06\x02\x00\x2f\x80\xce\x7f\xd6\x01\xbd\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb8\xb0\x45\x40\x80\xd6\x6a\xbc\xc8\x19\x6b\xbc\x50\x74\xc0\xc3\x88\x84\xb6\x42\x89\xc4\xd0\xc1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x82\x06\x24\x00\x2c\x80\xd0\x7f\xe7\x01\x82\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x1a\xce\x45\x40\x80\x4d\x74\xbc\x1f\xae\x73\xbc\xed\x0a\xbe\xc3\x5f\xaa\xb6\x42\x35\xb9\x8e\xc1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x79\x06\x10\x00\x2b\x80\xd1\x7f\x00\x02\x79\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x8a\xd2\x45\x40\x00\x12\x80\xbc\xff\x06\x80\xbc\x5a\xcc\xc0\xc3\x86\x81\xb6\x42\xd4\x42\x21\xc1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x05\xde\xff\x1b\x80\xe1\x7f\x05\x02\x04\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf9\x8c\x46\x40\x80\xf5\x80\xbc\x9e\x64\x81\xbc\x5b\x64\xbe\xc3\x9e\xac\xb6\x42\x9c\x60\x08\xc0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x44\x04\xeb\xff\x14\x80\xe9\x7f\xe9\x01\x43\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x91\xed\x46\x40\x80\x67\x74\xbc\xfb\xe4\x74\xbc\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3d\x0d\xee\xc0\xe9\x80\x13\xc1\x7b\x0d\xe8\x40\x9d\x94\xf5\x40\xc9\x1a\xc7\x41\x53\xd6\x0e\xc2\x69\xb0\xc6\x42\x9a\xbe\x57\x42\x86\x60\xa3\x44\xed\xa8\xa8\xc4\x3a\x3d\x2d\x45\xfc\xe9\x7a\x44\xc3\x3b\x2d\x42\xa2\x2d\x2d\x42\xd1\x4b\x2d\x42\x6e\x4b\x2d\x42\x19\x88\x5c\xb8\xde\x95\x48\x38\x8b\x3e\xaa\x39\xc3\xeb\xa6\x39\x3b\x8f\x85\xbc\xa0\xb9\x23\x3e\x78\x41\x2d\x42\x30\x7c\xba\xbd\x57\x12\x62\xbb\x32\x87\xbd\x3d\xf0\x30\xed\xbf\x61\x3f\xa3\xbd\x39\xd2\x9d\x40\x00\x00\x00\x80"
# d = b"\xe5\x07\x01\x12\x01\x00\x08\xcd\x7f\x68\xcf\x97\xbb\x60\x83\x97\x76\x42\x2d\x07\x00\x00\x00\xff\xd3\x3c\xbd\x43\x43\xd0\xd6\x42\x30\x67\xeb\xc3\x41\x05\x29\x42\xdb\x8e\x40\x3f\x65\xdd\x1f\x41\x8d\x7c\x02\x02\x6a\x1d\xa4\xe2\xb1\xfc\x8a\x7c\x17\x49\x61\x3c\xbb\xcd\xb0\xbe\xcc\xd3\x9d\xbd\x1f\x6e\xab\x3f\xc0\xed\xad\xbc\xac\xed\xd3\x3c\x12\x4d\xbc\xc3\xa3\xa9\xb6\x42\xf1\xd7\xc3\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x5a\x05\x14\x00\x1e\x80\xde\x7f\xf0\x01\x5a\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x7b\x62\x46\x40\x80\x5e\x78\xbc\x79\x28\x78\xbc\x0f\x0e\xbf\xc3\xfb\x80\xb6\x42\x60\x25\xb4\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb7\x06\x01\x00\x2f\x80\xcd\x7f\xf9\x01\xb6\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc3\xb3\x45\x40\x80\x54\x7c\xbc\x20\xa3\x7c\xbc\x84\xa5\xbc\xc3\x86\xad\xb6\x42\x99\xe4\xa3\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe5\x03\xe0\xff\x11\x80\xeb\x7f\x0c\x02\xe4\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe5\x1c\x47\x40\x00\xf2\x82\xbc\x7c\x40\x83\xbc\x1b\x67\xbf\xc3\xcd\x84\xb6\x42\x13\xf8\x93\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x07\x04\x00\x33\x80\xc9\x7f\x0c\x02\x00\x07\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x06\x8f\x45\x40\x40\x0b\x83\xbc\xcc\x2d\x83\xbc\xd2\xfe\xbc\xc3\x1a\xac\xb6\x42\x53\x19\x84\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0\x03\x0d\x00\x10\x80\xed\x7f\xf3\x01\xc0\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa4\x2f\x47\x40\x80\xc6\x79\xbc\x3d\xad\x79\xbc\x1f\xc0\xbf\xc3\xe9\x82\xb6\x42\x82\xf0\x67\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x34\x07\x17\x00\x35\x80\xc7\x7f\xe0\x01\x34\x07\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x27\x75\x45\x40\x00\xb5\x70\xbc\xb1\x6a\x70\xbc\x2d\x57\xbd\xc3\x2c\xa8\xb6\x42\xcc\x79\x47\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc8\x05\x04\x00\x23\x80\xd9\x7f\xf1\x01\xc8\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3e\x2b\x46\x40\x80\xc1\x78\xbc\xf7\xea\x78\xbc\x16\x19\xc0\xc3\x7b\x83\xb6\x42\xba\xff\x27\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0\x05\xc3\xff\x23\x80\xda\x7f\xee\x01\xbf\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x58\x2f\x46\x40\x80\x8b\x75\xbc\x8c\x2b\x77\xbc\x82\xb0\xbd\xc3\xba\xad\xb6\x42\x44\xcb\x07\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xbd\x06\x02\x00\x2f\x80\xce\x7f\xd6\x01\xbd\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb8\xb0\x45\x40\x80\xd6\x6a\xbc\xc8\x19\x6b\xbc\x50\x74\xc0\xc3\x88\x84\xb6\x42\x89\xc4\xd0\xc1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x82\x06\x24\x00\x2c\x80\xd0\x7f\xe7\x01\x82\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x1a\xce\x45\x40\x80\x4d\x74\xbc\x1f\xae\x73\xbc\xed\x0a\xbe\xc3\x5f\xaa\xb6\x42\x35\xb9\x8e\xc1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x79\x06\x10\x00\x2b\x80\xd1\x7f\x00\x02\x79\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x8a\xd2\x45\x40\x00\x12\x80\xbc\xff\x06\x80\xbc\x5a\xcc\xc0\xc3\x86\x81\xb6\x42\xd4\x42\x21\xc1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x05\xde\xff\x1b\x80\xe1\x7f\x05\x02\x04\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf9\x8c\x46\x40\x80\xf5\x80\xbc\x9e\x64\x81\xbc\x5b\x64\xbe\xc3\x9e\xac\xb6\x42\x9c\x60\x08\xc0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x44\x04\xeb\xff\x14\x80\xe9\x7f\xe9\x01\x43\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x91\xed\x46\x40\x80\x67\x74\xbc\xfb\xe4\x74\xbc\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x6a\x9c\x03\xc1\x1b\x45\x05\xc1\xc2\x2a\x51\x40\x79\x47\x83\x40\xd3\x58\x95\xc1\xbe\x16\x14\x41\x21\x24\xdf\x40\xe2\xe9\xa3\x41\x75\x94\xf0\xc2\x75\x6e\x00\x44\xaa\xdf\x47\x44\xce\x99\xcf\x44\xd0\xb0\x2d\x42\xeb\xa8\x2d\x42\xea\xc4\x2d\x42\xef\xc1\x2d\x42\x9d\xf4\x4f\xb9\x1a\xae\xab\x37\x89\xa7\x95\x39\x33\x83\x94\x39\x10\x00\x1d\xbc\x00\xe5\x8f\x3d\x0f\xb5\x2d\x42\xc2\x0e\xf0\xbc\x89\xef\x4f\xba\xeb\x12\x93\xbd\xe3\x1c\xf6\xbf\xcb\x4a\x97\xbd\x97\xab\x36\xbf\x00\x00\x00\x80"
# d = b"\xe5\x07\x01\x12\x01\x00\x08\xcd\x7f\x68\xcf\x97\xbb\x60\x47\x42\x77\x42\x32\x07\x00\x00\x00\xff\xdd\xbc\xc0\x43\xbd\x33\xd7\x42\xfb\x92\xea\xc3\x7f\xd4\x26\x42\x10\x63\xcc\x3f\x5d\x1a\x1e\x41\x7b\x7c\x2e\x04\x7e\x1d\x99\xe2\xed\xfc\x88\x7c\xf3\x2a\xec\x3c\xbd\x7f\xa9\xbe\x8e\xcd\x8a\x3e\xeb\x5e\xab\x3f\xc0\xf5\x18\xbd\x5f\xe2\xc4\x3c\x12\x4d\xbc\xc3\xa3\xa9\xb6\x42\xf1\xd7\xc3\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x5a\x05\x14\x00\x1e\x80\xde\x7f\xf0\x01\x5a\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x7b\x62\x46\x40\x80\x5e\x78\xbc\x79\x28\x78\xbc\x0f\x0e\xbf\xc3\xfb\x80\xb6\x42\x60\x25\xb4\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb7\x06\x01\x00\x2f\x80\xcd\x7f\xf9\x01\xb6\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc3\xb3\x45\x40\x80\x54\x7c\xbc\x20\xa3\x7c\xbc\x84\xa5\xbc\xc3\x86\xad\xb6\x42\x99\xe4\xa3\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe5\x03\xe0\xff\x11\x80\xeb\x7f\x0c\x02\xe4\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe5\x1c\x47\x40\x00\xf2\x82\xbc\x7c\x40\x83\xbc\x1b\x67\xbf\xc3\xcd\x84\xb6\x42\x13\xf8\x93\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x07\x04\x00\x33\x80\xc9\x7f\x0c\x02\x00\x07\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x06\x8f\x45\x40\x40\x0b\x83\xbc\xcc\x2d\x83\xbc\xd2\xfe\xbc\xc3\x1a\xac\xb6\x42\x53\x19\x84\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0\x03\x0d\x00\x10\x80\xed\x7f\xf3\x01\xc0\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa4\x2f\x47\x40\x80\xc6\x79\xbc\x3d\xad\x79\xbc\x1f\xc0\xbf\xc3\xe9\x82\xb6\x42\x82\xf0\x67\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x34\x07\x17\x00\x35\x80\xc7\x7f\xe0\x01\x34\x07\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x27\x75\x45\x40\x00\xb5\x70\xbc\xb1\x6a\x70\xbc\x2d\x57\xbd\xc3\x2c\xa8\xb6\x42\xcc\x79\x47\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc8\x05\x04\x00\x23\x80\xd9\x7f\xf1\x01\xc8\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3e\x2b\x46\x40\x80\xc1\x78\xbc\xf7\xea\x78\xbc\x16\x19\xc0\xc3\x7b\x83\xb6\x42\xba\xff\x27\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0\x05\xc3\xff\x23\x80\xda\x7f\xee\x01\xbf\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x58\x2f\x46\x40\x80\x8b\x75\xbc\x8c\x2b\x77\xbc\x82\xb0\xbd\xc3\xba\xad\xb6\x42\x44\xcb\x07\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xbd\x06\x02\x00\x2f\x80\xce\x7f\xd6\x01\xbd\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb8\xb0\x45\x40\x80\xd6\x6a\xbc\xc8\x19\x6b\xbc\x50\x74\xc0\xc3\x88\x84\xb6\x42\x89\xc4\xd0\xc1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x82\x06\x24\x00\x2c\x80\xd0\x7f\xe7\x01\x82\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x1a\xce\x45\x40\x80\x4d\x74\xbc\x1f\xae\x73\xbc\xed\x0a\xbe\xc3\x5f\xaa\xb6\x42\x35\xb9\x8e\xc1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x79\x06\x10\x00\x2b\x80\xd1\x7f\x00\x02\x79\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x8a\xd2\x45\x40\x00\x12\x80\xbc\xff\x06\x80\xbc\x5a\xcc\xc0\xc3\x86\x81\xb6\x42\xd4\x42\x21\xc1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x05\xde\xff\x1b\x80\xe1\x7f\x05\x02\x04\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf9\x8c\x46\x40\x80\xf5\x80\xbc\x9e\x64\x81\xbc\x5b\x64\xbe\xc3\x9e\xac\xb6\x42\x9c\x60\x08\xc0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x44\x04\xeb\xff\x14\x80\xe9\x7f\xe9\x01\x43\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x91\xed\x46\x40\x80\x67\x74\xbc\xfb\xe4\x74\xbc\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x76\x18\xd3\xc0\x7b\xa2\x90\xc0\x1a\x1e\xe8\x40\x7d\xca\xc0\x40\x09\x0c\xb3\xc2\x95\x25\x80\xc0\x9d\xfa\x48\xc1\xea\x86\x63\xc0\xb0\xc8\x61\xc5\xf6\xa3\x01\xc4\x9e\x91\xc6\xc4\x5e\x54\x05\xc4\x8b\x8e\x2b\x42\x7a\x8b\x2b\x42\x66\xa0\x2b\x42\x23\x99\x2b\x42\xce\x97\x33\xb9\x85\x26\x96\xb8\x48\xfd\x8e\x39\x30\x60\x94\x39\x80\x45\xb4\x3b\x30\xc0\x47\x3e\xc5\x90\x2b\x42\xd8\xaa\x25\x3d\x7e\x78\x57\xbb\xf0\x95\x86\x3c\x2f\xef\xa9\xbf\x43\x3b\x11\xbe\x20\xf9\xa6\x3e\x00\x00\x00\x80"

# Packet ID 10, len 882
# d = b"\xe5\x07\x01\x12\x01\x0a\x08\xcd\x7f\x68\xcf\x97\xbb\x60\x58\x8a\x96\x42\xc5\x08\x00\x00\x00\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"

# Packet ID 1, len 625
# d = b"\xe5\x07\x01\x12\x01\x01\x08\xcd\x7f\x68\xcf\x97\xbb\x60\xbd\xc1\x98\x42\xe5\x08\x00\x00\x00\xff\x00\x23\x1c\xc8\x20\x15\x0d\x03\x00\x00\x00\x58\x02\x50\x00\x00\xff\x00\x11\xb1\x4c\x74\x3f\x00\x63\x47\xdc\x3d\x00\xfd\xe0\x30\x3e\x00\x78\xee\x81\x3e\x00\x70\x27\xa4\x3e\x00\x67\x6f\xb4\x3e\x00\x38\x9a\xc1\x3e\x00\xc0\xfb\xd2\x3e\x00\x86\x01\xe4\x3e\x00\xef\x84\xf7\x3e\x00\x0e\x66\x03\x3f\x00\xe2\x34\x20\x3f\x00\x7d\x3d\x29\x3f\x00\x4a\x8f\x31\x3f\x00\x7e\x01\x40\x3f\x00\x02\x69\x4d\x3f\x00\x5c\xfd\x63\x3f\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00"


# FROM F1 DIRECTLY
# d = b'\xe5\x07\x01\x12\x01\x02\xebH\xa9k\x85\xf5k\xaa\xfc4\xc0A\xc9\x05\x00\x00\x00\xff\x00\x00\x00\x00\x8e\x03\x00\x00\x00\x00\x00\x00\x10E\x98B\x10E\x98B\x00\x00\x00\x80\x01\x01\x00\x00\x00\x01\x00\x00\x00\x00\x01\x01\x02\x00\x00\x00\x00\x00\x00\xe5p\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf7\xc6kC\xf7\xc6kC\x00\x00\x00\x80\x01\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xde\xb7cC\xde\xb7cC\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf3\xcd[C\xf3\xcd[C\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x15\xb4SC\x15\xb4SC\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x03LC@\x03LC\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb5\xc0CC\xb5\xc0CC\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc1\xbc;C\xc1\xbc;C\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0\xc53C\xf0\xc53C\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa6\xc1+C\xa6\xc1+C\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00=\xe6#C=\xe6#C\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00Y\xfe\x1bCY\xfe\x1bC\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00}\xd7\x13C}\xd7\x13C\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb3\xee\x0bC\xb3\xee\x0bC\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
# d = b'\xe5\x07\x01\x12\x01\x0b\xebH\xa9k\x85\xf5k\xaa\x00Z\xc0A\xca\x05\x00\x00\x00\xff\x02\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0f\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\x11\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [12]:
# g = PacketHeader.from_buffer_copy(d)



# print("Offsets")
# print('m_packet_format: ', PacketHeader.m_packet_format.offset)
# print('m_game_major_version: ', PacketHeader.m_game_major_version.offset)
# print('m_game_minor_version: ', PacketHeader.m_game_minor_version.offset)
# print('m_packet_version: ', PacketHeader.m_packet_version.offset)
# print('m_packet_id: ', PacketHeader.m_packet_id.offset)
# print('m_session_uid: ', PacketHeader.m_session_uid.offset)
# print('m_session_time: ', PacketHeader.m_session_time.offset)
# print('m_frame_identifier: ', PacketHeader.m_frame_identifier.offset)
# print('m_player_car_index: ', PacketHeader.m_player_car_index.offset)
# print('m_secondary_player_car_index: ', PacketHeader.m_secondary_player_car_index.offset)
# print("")

# print('Sizes')
# print('m_packet_format: ', PacketHeader.m_packet_format.size)
# print('m_game_major_version: ', PacketHeader.m_game_major_version.size)
# print('m_game_minor_version: ', PacketHeader.m_game_minor_version.size)
# print('m_packet_version: ', PacketHeader.m_packet_version.size)
# print('m_packet_id: ', PacketHeader.m_packet_id.size)
# print('m_session_uid: ', PacketHeader.m_session_uid.size)
# print('m_session_time: ', PacketHeader.m_session_time.size)
# print('m_frame_identifier: ', PacketHeader.m_frame_identifier.size)
# print('m_player_car_index: ', PacketHeader.m_player_car_index.size)
# print('m_secondary_player_car_index: ', PacketHeader.m_secondary_player_car_index.size)







In [13]:
# HEADER

d = data[0]

m_header = PacketHeader.from_buffer_copy(d)
print("m_packet_format: ", m_header.m_packet_format)
print("m_game_major_version: ", m_header.m_game_major_version)
print("m_game_minor_version: ", m_header.m_game_minor_version)
print("m_packet_version: ", m_header.m_packet_version)
print("m_packet_id: ", m_header.m_packet_id)
print("m_session_uid: ", m_header.m_session_uid)
print("m_session_time: ", m_header.m_session_time)
print("m_frame_identifier: ", m_header.m_frame_identifier)
print("m_player_car_index: ", m_header.m_player_car_index)
print("m_secondary_player_car_index: ", m_header.m_secondary_player_car_index)

m_packet_format:  2021
m_game_major_version:  1
m_game_minor_version:  18
m_packet_version:  1
m_packet_id:  0
m_session_uid:  18401439455031850975
m_session_time:  2.1630443095317876e-41
m_frame_identifier:  4278190080
m_player_car_index:  251
m_secondary_player_car_index:  32


In [ ]:
# CAR MOTION

m_motion = CarMotionData.from_buffer_copy(d[24:])
m_world_position_x = m_motion.m_world_position_x
m_world_position_y = m_motion.m_world_position_y
m_world_position_z = m_motion.m_world_position_z
m_world_velocity_x = m_motion.m_world_velocity_x
m_world_velocity_y = m_motion.m_world_velocity_y
m_world_velocity_z = m_motion.m_world_velocity_z
m_world_forward_dir_x = m_motion.m_world_forward_dir_x
m_world_forward_dir_y = m_motion.m_world_forward_dir_y
m_world_forward_dir_z = m_motion.m_world_forward_dir_z
m_world_right_dir_x = m_motion.m_world_right_dir_x
m_world_right_dir_y = m_motion.m_world_right_dir_y
m_world_right_dir_z = m_motion.m_world_right_dir_z
m_g_force_lateral = m_motion.m_g_force_lateral
m_g_force_longitudinal = m_motion.m_g_force_longitudinal
m_g_force_vertical = m_motion.m_g_force_vertical
m_yaw = m_motion.m_yaw
m_pitch = m_motion.m_pitch
m_roll = m_motion.m_roll

print('m_world_position_x: ', m_world_position_x)
print('m_world_position_y: ', m_world_position_y)
print('m_world_position_z: ', m_world_position_z)
print('m_world_velocity_x: ', m_world_velocity_x)
print('m_world_velocity_y: ', m_world_velocity_y)
print('m_world_velocity_z: ', m_world_velocity_z)
print('m_world_forward_dir_x: ', m_world_forward_dir_x)
print('m_world_forward_dir_y: ', m_world_forward_dir_y)
print('m_world_forward_dir_z: ', m_world_forward_dir_z)
print('m_world_right_dir_x: ', m_world_right_dir_x)
print('m_world_right_dir_y: ', m_world_right_dir_y)
print('m_world_right_dir_z: ', m_world_right_dir_z)
print('m_g_force_lateral: ', m_g_force_lateral)
print('m_g_force_longitudinal: ', m_g_force_longitudinal)
print('m_g_force_vertical: ', m_g_force_vertical)
print('m_yaw: ', m_yaw)
print('m_pitch: ', m_pitch)
print('m_roll: ', m_roll)

In [ ]:
# MARSHAL ZONE
m_marshal = MarshalZone.from_buffer_copy(d[24:])
m_zone_start = m_marshal.m_zone_start        # Fraction (0..1) of way through the lap the marshal zone starts
m_zone_flag = m_marshal.m_zone_flag          # -1 = invalid/unknown, 0 = none, 1 = green, 2 = blue, 3 = yellow, 4 = red

print('m_zone_start: ', m_zone_start)
print('m_zone_flag: ', m_zone_flag)

In [216]:
# d = b"\xe5\x07\x01\x12\x01\x02\x08\xcd\x7f\x68\xcf\x97\xbb\x60\x07\x24\x8c\x42\x29\x08\x00\x00\x00\xff\x00\x00\x00\x00\x86\x88\x00\x00\x00\x00\x00\x00\x01\x85\xc4\x44\x01\x85\xc4\x44\x00\x00\x00\x80\x01\x01\x00\x00\x00\x01\x00\x00\x00\x00\x01\x01\x02\x00\x00\x00\x00\x00\x00\xe5\x70\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x74\xe3\x6b\x43\x74\xe3\x6b\x43\x00\x00\x00\x80\x01\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x92\xd1\x63\x43\x92\xd1\x63\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x40\xe6\x5b\x43\x40\xe6\x5b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xb6\x53\x43\xcd\xb6\x53\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfd\x4b\x43\x00\xfd\x4b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc2\xb2\x43\x43\xc2\xb2\x43\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x52\xcb\x3b\x43\x52\xcb\x3b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x6c\xb2\x33\x43\x6c\xb2\x33\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11\xdc\x2b\x43\x11\xdc\x2b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb2\xc6\x23\x43\xb2\xc6\x23\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb9\xbc\x1b\x43\xb9\xbc\x1b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3d\xbe\x13\x43\x3d\xbe\x13\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x62\x03\x0c\x43\x62\x03\x0c\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"
# d = b"\xe5\x07\x01\x12\x01\x02\x08\xcd\x7f\x68\xcf\x97\xbb\x60\x14\x35\x8c\x42\x2a\x08\x00\x00\x00\xff\x00\x00\x00\x00\xa7\x88\x00\x00\x00\x00\x00\x00\x19\x88\xc4\x44\x19\x88\xc4\x44\x00\x00\x00\x80\x01\x01\x00\x00\x00\x01\x00\x00\x00\x00\x01\x01\x02\x00\x00\x00\x00\x00\x00\xe5\x70\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x74\xe3\x6b\x43\x74\xe3\x6b\x43\x00\x00\x00\x80\x01\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x92\xd1\x63\x43\x92\xd1\x63\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x40\xe6\x5b\x43\x40\xe6\x5b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xb6\x53\x43\xcd\xb6\x53\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfd\x4b\x43\x00\xfd\x4b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc2\xb2\x43\x43\xc2\xb2\x43\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x52\xcb\x3b\x43\x52\xcb\x3b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x6c\xb2\x33\x43\x6c\xb2\x33\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11\xdc\x2b\x43\x11\xdc\x2b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb2\xc6\x23\x43\xb2\xc6\x23\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb9\xbc\x1b\x43\xb9\xbc\x1b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3d\xbe\x13\x43\x3d\xbe\x13\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x62\x03\x0c\x43\x62\x03\x0c\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"
# d = b"\xe5\x07\x01\x12\x01\x02\x08\xcd\x7f\x68\xcf\x97\xbb\x60\x37\x57\x8c\x42\x2c\x08\x00\x00\x00\xff\x00\x00\x00\x00\xea\x88\x00\x00\x00\x00\x00\x00\x92\x8e\xc4\x44\x92\x8e\xc4\x44\x00\x00\x00\x80\x01\x01\x00\x00\x00\x01\x00\x00\x00\x00\x01\x01\x02\x00\x00\x00\x00\x00\x00\xe5\x70\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x74\xe3\x6b\x43\x74\xe3\x6b\x43\x00\x00\x00\x80\x01\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x92\xd1\x63\x43\x92\xd1\x63\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x40\xe6\x5b\x43\x40\xe6\x5b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xb6\x53\x43\xcd\xb6\x53\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfd\x4b\x43\x00\xfd\x4b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc2\xb2\x43\x43\xc2\xb2\x43\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x52\xcb\x3b\x43\x52\xcb\x3b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x6c\xb2\x33\x43\x6c\xb2\x33\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11\xdc\x2b\x43\x11\xdc\x2b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb2\xc6\x23\x43\xb2\xc6\x23\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb9\xbc\x1b\x43\xb9\xbc\x1b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3d\xbe\x13\x43\x3d\xbe\x13\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x62\x03\x0c\x43\x62\x03\x0c\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"
# # d = b"\xe5\x07\x01\x12\x01\x02\x08\xcd\x7f\x68\xcf\x97\xbb\x60\x6b\x8a\x8c\x42\x2f\x08\x00\x00\x00\xff\x00\x00\x00\x00\x4e\x89\x00\x00\x00\x00\x00\x00\x14\x99\xc4\x44\x14\x99\xc4\x44\x00\x00\x00\x80\x01\x01\x00\x00\x00\x01\x00\x00\x00\x00\x01\x01\x02\x00\x00\x00\x00\x00\x00\xe5\x70\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x74\xe3\x6b\x43\x74\xe3\x6b\x43\x00\x00\x00\x80\x01\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x92\xd1\x63\x43\x92\xd1\x63\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x40\xe6\x5b\x43\x40\xe6\x5b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xb6\x53\x43\xcd\xb6\x53\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfd\x4b\x43\x00\xfd\x4b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc2\xb2\x43\x43\xc2\xb2\x43\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x52\xcb\x3b\x43\x52\xcb\x3b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x6c\xb2\x33\x43\x6c\xb2\x33\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11\xdc\x2b\x43\x11\xdc\x2b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb2\xc6\x23\x43\xb2\xc6\x23\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb9\xbc\x1b\x43\xb9\xbc\x1b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3d\xbe\x13\x43\x3d\xbe\x13\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x62\x03\x0c\x43\x62\x03\x0c\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"
# d = b"\xe5\x07\x01\x12\x01\x02\x08\xcd\x7f\x68\xcf\x97\xbb\x60\xb0\x20\x7b\x42\x4f\x07\x00\x00\x00\xff\x00\x00\x00\x00\x0d\x6c\x00\x00\x00\x00\x00\x00\x9d\x39\xc1\x44\x9d\x39\xc1\x44\x00\x00\x00\x80\x01\x01\x00\x00\x00\x01\x00\x00\x00\x00\x01\x01\x02\x00\x00\x00\x00\x00\x00\xe5\x70\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x74\xe3\x6b\x43\x74\xe3\x6b\x43\x00\x00\x00\x80\x01\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x92\xd1\x63\x43\x92\xd1\x63\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x40\xe6\x5b\x43\x40\xe6\x5b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xb6\x53\x43\xcd\xb6\x53\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfd\x4b\x43\x00\xfd\x4b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc2\xb2\x43\x43\xc2\xb2\x43\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x52\xcb\x3b\x43\x52\xcb\x3b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x6c\xb2\x33\x43\x6c\xb2\x33\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11\xdc\x2b\x43\x11\xdc\x2b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb2\xc6\x23\x43\xb2\xc6\x23\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb9\xbc\x1b\x43\xb9\xbc\x1b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3d\xbe\x13\x43\x3d\xbe\x13\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x62\x03\x0c\x43\x62\x03\x0c\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"
# d = b"\xe5\x07\x01\x12\x01\x02\x08\xcd\x7f\x68\xcf\x97\xbb\x60\xd6\xec\x6b\x42\xdd\x06\x00\x00\x00\xff\x00\x00\x00\x00\x35\x5d\x00\x00\x00\x00\x00\x00\xd8\xd0\xad\x44\xd8\xd0\xad\x44\x00\x00\x00\x80\x01\x01\x00\x00\x00\x01\x00\x00\x00\x00\x01\x01\x02\x00\x00\x00\x00\x00\x00\xe5\x70\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x74\xe3\x6b\x43\x74\xe3\x6b\x43\x00\x00\x00\x80\x01\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x92\xd1\x63\x43\x92\xd1\x63\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x40\xe6\x5b\x43\x40\xe6\x5b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xb6\x53\x43\xcd\xb6\x53\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfd\x4b\x43\x00\xfd\x4b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc2\xb2\x43\x43\xc2\xb2\x43\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x52\xcb\x3b\x43\x52\xcb\x3b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x6c\xb2\x33\x43\x6c\xb2\x33\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11\xdc\x2b\x43\x11\xdc\x2b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb2\xc6\x23\x43\xb2\xc6\x23\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb9\xbc\x1b\x43\xb9\xbc\x1b\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3d\xbe\x13\x43\x3d\xbe\x13\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x62\x03\x0c\x43\x62\x03\x0c\x43\x00\x00\x00\x80\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"


In [ ]:
# LAP DATA
m_lapdata = LapData.from_buffer_copy(d[24:]
                                     )
m_last_lap_time_in_ms = m_lapdata.m_last_lap_time_in_ms                                          # Last lap time in milliseconds
m_current_lap_time_in_ms = m_lapdata.m_current_lap_time_in_ms                                    # Current time around the lap in milliseconds
m_sector1_time_in_ms = m_lapdata.m_sector1_time_in_ms                                            # Sector 1 time in milliseconds
m_sector2_time_in_ms = m_lapdata.m_sector2_time_in_ms                                            # Sector 2 time in milliseconds
m_lap_distance = m_lapdata.m_lap_distance                                                        # Distance vehicle is around current lap in metres – could        # be negative if line hasn’t been crossed yet
m_total_distance = m_lapdata.m_total_distance                                                    # Total distance travelled in session in metres – could        # be negative if line hasn’t been crossed yet
m_safety_car_delta = m_lapdata.m_safety_car_delta                                                # Delta in seconds for safety car
m_car_position = m_lapdata.m_car_position                                                        # Car race position
m_current_lap_num = m_lapdata.m_current_lap_num                                                  # Current lap number
m_pit_status = m_lapdata.m_pit_status                                                            # 0 = none, 1 = pitting, 2 = in pit area
m_num_pit_stops = m_lapdata.m_num_pit_stops                                                      # Number of pit stops taken in this race
m_sector = m_lapdata.m_sector                                                                    # 0 = sector1, 1 = sector2, 2 = sector3
m_current_lap_invalid = m_lapdata.m_current_lap_invalid                                          # Current lap invalid - 0 = valid, 1 = invalid
m_penalties = m_lapdata.m_penalties                                                              # Accumulated time penalties in seconds to be added
m_warnings = m_lapdata.m_warnings                                                                # Accumulated number of warnings issued
m_num_unserved_drive_through_pens = m_lapdata.m_num_unserved_drive_through_pens                  # Num drive through pens left to serve
m_num_unserved_stop_go_pens = m_lapdata.m_num_unserved_stop_go_pens                              # Num stop go pens left to serve
m_grid_position = m_lapdata.m_grid_position                                                      # Grid position the vehicle started the race in
m_driver_status = m_lapdata.m_driver_status                                                      # Status of driver - 0 = in garage, 1 = flying lap        # 2 = in lap, 3 = out lap, 4 = on track
m_result_status = m_lapdata.m_result_status                                                      # Result status - 0 = invalid, 1 = inactive, 2 = active        # 3 = finished, 4 = didnotfinish, 5 = disqualified        # 6 = not classified, 7 = retired
m_pit_lane_timer_active = m_lapdata.m_pit_lane_timer_active                                      # Pit lane timing, 0 = inactive, 1 = active
m_pit_lane_time_in_lane_in_ms = m_lapdata.m_pit_lane_time_in_lane_in_ms                          # If active, the current time spent in the pit lane in ms
m_pit_stop_timer_in_ms = m_lapdata.m_pit_stop_timer_in_ms                                        # Time of the actual pit stop in ms
m_pit_stop_should_serve_pen = m_lapdata.m_pit_stop_should_serve_pen                              # Whether the car should serve a penalty at this stop

print('m_last_lap_time_in_ms: ', m_lapdata.m_last_lap_time_in_ms)
print('m_current_lap_time_in_ms: ', m_lapdata.m_current_lap_time_in_ms)
print('m_sector1_time_in_ms: ', m_lapdata.m_sector1_time_in_ms)
print('m_sector2_time_in_ms: ', m_lapdata.m_sector2_time_in_ms)
print('m_lap_distance: ', m_lapdata.m_lap_distance)
print('m_total_distance: ', m_lapdata.m_total_distance)
print('m_safety_car_delta: ', m_lapdata.m_safety_car_delta)
print('m_car_position: ', m_lapdata.m_car_position)
print('m_current_lap_num: ', m_lapdata.m_current_lap_num)
print('m_pit_status: ', m_lapdata.m_pit_status)
print('m_num_pit_stops: ', m_lapdata.m_num_pit_stops)
print('m_sector: ', m_lapdata.m_sector)
print('m_current_lap_invalid: ', m_lapdata.m_current_lap_invalid)
print('m_penalties: ', m_lapdata.m_penalties)
print('m_warnings: ', m_lapdata.m_warnings)
print('m_num_unserved_drive_through_pens: ', m_lapdata.m_num_unserved_drive_through_pens)
print('m_num_unserved_stop_go_pens: ', m_lapdata.m_num_unserved_stop_go_pens)
print('m_grid_position: ', m_lapdata.m_grid_position)
print('m_driver_status: ', m_lapdata.m_driver_status)
print('m_result_status: ', m_lapdata.m_result_status)
print('m_pit_lane_timer_active: ', m_lapdata.m_pit_lane_timer_active)
print('m_pit_lane_time_in_lane_in_ms: ', m_lapdata.m_pit_lane_time_in_lane_in_ms)
print('m_pit_stop_timer_in_ms: ', m_lapdata.m_pit_stop_timer_in_ms)
print('m_pit_stop_should_serve_pen: ', m_lapdata.m_pit_stop_should_serve_pen)

In [ ]:
# CAR STATUS DATA
m_carstatus = CarStatusData.from_buffer_copy(d[24:])       
m_traction_control = m_carstatus.m_traction_control                                      # Traction control - 0 = off, 1 = medium, 2 = full
m_anti_lock_brakes = m_carstatus.m_anti_lock_brakes                                      # 0 (off) - 1 (on)
m_fuel_mix = m_carstatus.m_fuel_mix                                                      # Fuel mix - 0 = lean, 1 = standard, 2 = rich, 3 = max
m_front_brake_bias = m_carstatus.m_front_brake_bias                                      # Front brake bias (percentage)
m_pit_limiter_status = m_carstatus.m_pit_limiter_status                                  # Pit limiter status - 0 = off, 1 = on
m_fuel_in_tank = m_carstatus.m_fuel_in_tank                                              # Current fuel mass
m_fuel_capacity = m_carstatus.m_fuel_capacity                                            # Fuel capacity
m_fuel_remaining_laps = m_carstatus.m_fuel_remaining_laps                                # Fuel remaining in terms of laps (value on MFD)
m_max_rpm = m_carstatus.m_max_rpm                                                        # Cars max RPM, point of rev limiter
m_idle_rpm = m_carstatus.m_idle_rpm                                                      # Cars idle RPM
m_max_gears = m_carstatus.m_max_gears                                                    # Maximum number of gears
m_drs_allowed = m_carstatus.m_drs_allowed                                                # 0 = not allowed, 1 = allowed
m_drs_activation_distance = m_carstatus.m_drs_activation_distance                        # 0 = DRS not available, non-zero - DRS will be available        # in [X] metres
m_actual_tyre_compound = m_carstatus.m_actual_tyre_compound                              # F1 Modern - 16 = C5, 17 = C4, 18 = C3, 19 = C2, 20 = C1        # 7 = inter, 8 = wet        # F1 Classic - 9 = dry, 10 = wet        # F2 – 11 = super soft, 12 = soft, 13 = medium, 14 = hard        # 15 = wet
m_visual_tyre_compound = m_carstatus.m_visual_tyre_compound                              # F1 visual (can be different from actual compound)        # 16 = soft, 17 = medium, 18 = hard, 7 = inter, 8 = wet        # F1 Classic – same as above        # F2 ‘19, 15 = wet, 19 – super soft, 20 = soft        # 21 = medium , 22 = hard
m_tyres_age_laps = m_carstatus.m_tyres_age_laps                                          # Age in laps of the current set of tyres
m_vehicle_fia_flags = m_carstatus.m_vehicle_fia_flags                                    # -1 = invalid/unknown, 0 = none, 1 = green        # 2 = blue, 3 = yellow, 4 = red
m_ers_store_energy = m_carstatus.m_ers_store_energy                                      # ERS energy store in Joules
m_ers_deploy_mode = m_carstatus.m_ers_deploy_mode                                        # ERS deployment mode, 0 = none, 1 = medium        # 2 = hotlap, 3 = overtake
m_ers_harvested_this_lap_mguk = m_carstatus.m_ers_harvested_this_lap_mguk                # ERS energy harvested this lap by MGU-K
m_ers_harvested_this_lap_mguh = m_carstatus.m_ers_harvested_this_lap_mguh                # ERS energy harvested this lap by MGU-H
m_ers_deployed_this_lap = m_carstatus.m_ers_deployed_this_lap                            # ERS energy deployed this lap
m_network_paused = m_carstatus.m_network_paused                                          # Whether the car is paused in a network game

print('m_traction_control: ', m_carstatus.m_traction_control)
print('m_anti_lock_brakes: ', m_carstatus.m_anti_lock_brakes)
print('m_fuel_mix: ', m_carstatus.m_fuel_mix)
print('m_front_brake_bias: ', m_carstatus.m_front_brake_bias)
print('m_pit_limiter_status: ', m_carstatus.m_pit_limiter_status)
print('m_fuel_in_tank: ', m_carstatus.m_fuel_in_tank)
print('m_fuel_capacity: ', m_carstatus.m_fuel_capacity)
print('m_fuel_remaining_laps: ', m_carstatus.m_fuel_remaining_laps)
print('m_max_rpm: ', m_carstatus.m_max_rpm)
print('m_idle_rpm: ', m_carstatus.m_idle_rpm)
print('m_max_gears: ', m_carstatus.m_max_gears)
print('m_drs_allowed: ', m_carstatus.m_drs_allowed)
print('m_drs_activation_distance: ', m_carstatus.m_drs_activation_distance)
print('m_actual_tyre_compound: ', m_carstatus.m_actual_tyre_compound)
print('m_visual_tyre_compound: ', m_carstatus.m_visual_tyre_compound)
print('m_tyres_age_laps: ', m_carstatus.m_tyres_age_laps)
print('m_vehicle_fia_flags: ', m_carstatus.m_vehicle_fia_flags)
print('m_ers_store_energy: ', m_carstatus.m_ers_store_energy)
print('m_ers_deploy_mode: ', m_carstatus.m_ers_deploy_mode)
print('m_ers_harvested_this_lap_mguk: ', m_carstatus.m_ers_harvested_this_lap_mguk)
print('m_ers_harvested_this_lap_mguh: ', m_carstatus.m_ers_harvested_this_lap_mguh)
print('m_ers_deployed_this_lap: ', m_carstatus.m_ers_deployed_this_lap)
print('m_network_paused: ', m_carstatus.m_network_paused)

In [ ]:
# CAR TELEMETRY

m_cartelem = CarTelemetryData.from_buffer_copy(d[24:])


m_speed = m_cartelem.m_speed                                                                    # Speed of car in kilometres per hour
m_throttle = m_cartelem.m_throttle                                                              # Amount of throttle applied (0.0 to 1.0)
m_steer = m_cartelem.m_steer                                                                    # Steering (-1.0 (full lock left) to 1.0 (full lock right))
m_brake = m_cartelem.m_brake                                                                    # Amount of brake applied (0.0 to 1.0)
m_clutch = m_cartelem.m_clutch                                                                  # Amount of clutch applied (0 to 100)
m_gear = m_cartelem.m_gear                                                                      # Gear selected (1-8, N=0, R=-1)
m_engine_rpm = m_cartelem.m_engine_rpm                                                          # Engine RPM
m_drs = m_cartelem.m_drs                                                                        # 0 = off, 1 = on
m_rev_lights_percent = m_cartelem.m_rev_lights_percent                                          # Rev lights indicator (percentage)
m_rev_lights_bit_value = m_cartelem.m_rev_lights_bit_value                                      # Rev lights (bit 0 = leftmost LED, bit 14 = rightmost LED)
m_brakes_temperature = m_cartelem.m_brakes_temperature                                          # Brakes temperature (celsius)
m_tyres_surface_temperature = m_cartelem.m_tyres_surface_temperature                            # Tyres surface temperature (celsius)
m_tyres_inner_temperature = m_cartelem.m_tyres_inner_temperature                                # Tyres inner temperature (celsius)
m_engine_temperature = m_cartelem.m_engine_temperature                                          # Engine temperature (celsius)
m_tyres_pressure = m_cartelem.m_tyres_pressure                                                  # Tyres pressure (PSI)
m_surface_type = m_cartelem.m_surface_type                                                      # Driving surface, see appendices

print('m_speed: ', m_speed)
print('m_throttle: ', m_throttle)
print('m_steer: ', m_steer)
print('m_brake: ', m_brake)
print('m_clutch: ', m_clutch)
print('m_gear: ', m_gear)
print('m_engine_rpm: ', m_engine_rpm)
print('m_drs: ', m_drs)
print('m_rev_lights_percent: ', m_rev_lights_percent)
print('m_rev_lights_bit_value: ', m_rev_lights_bit_value)
print('m_brakes_temperature: ', m_brakes_temperature)
print('m_tyres_surface_temperature: ', m_tyres_surface_temperature)
print('m_tyres_inner_temperature: ', m_tyres_inner_temperature)
print('m_engine_temperature: ', m_engine_temperature)
print('m_tyres_pressure: ', m_tyres_pressure)
print('m_surface_type: ', m_surface_type)

In [14]:
#  HEADER LOOP
df_header = pd.DataFrame()

ls_m_packet_format = []
ls_m_game_major_version = []
ls_m_game_minor_version = []
ls_m_packet_version = []
ls_m_packet_id = []
ls_m_session_uid = []
ls_m_session_time = []
ls_m_frame_identifier = []
ls_m_player_car_index = []
ls_m_secondary_player_car_index = []

for i in range(0, len(data)):
    # HEADER


    m_header = PacketHeader.from_buffer_copy(data[i])
    m_packet_format =  m_header.m_packet_format
    m_game_major_version =  m_header.m_game_major_version
    m_game_minor_version =  m_header.m_game_minor_version
    m_packet_version =  m_header.m_packet_version
    m_packet_id =  m_header.m_packet_id
    m_session_uid =  m_header.m_session_uid
    m_session_time =  m_header.m_session_time
    m_frame_identifier =  m_header.m_frame_identifier
    m_player_car_index =  m_header.m_player_car_index
    m_secondary_player_car_index =  m_header.m_secondary_player_car_index
    
    ls_m_packet_format.append(m_packet_format)
    ls_m_game_major_version.append(m_game_major_version)
    ls_m_game_minor_version.append(m_game_minor_version)
    ls_m_packet_version.append(m_packet_version)
    ls_m_packet_id.append(m_packet_id)
    ls_m_session_uid.append(m_session_uid)
    ls_m_session_time.append(m_session_time)
    ls_m_frame_identifier.append(m_frame_identifier)
    ls_m_player_car_index.append(m_player_car_index)
    ls_m_secondary_player_car_index.append(m_secondary_player_car_index)
    
df_header['m_packet_format'] = ls_m_packet_format
df_header['m_game_major_version'] = ls_m_game_major_version
df_header['m_game_minor_version'] = ls_m_game_minor_version
df_header['m_packet_version'] = ls_m_packet_version
df_header['m_packet_id'] = ls_m_packet_id
df_header['m_session_uid'] = ls_m_session_uid
df_header['m_session_time'] = ls_m_session_time
df_header['m_frame_identifier'] = ls_m_frame_identifier
df_header['m_player_car_index'] = ls_m_player_car_index
df_header['m_secondary_player_car_index'] = ls_m_secondary_player_car_index

df_header
# df_header.to_csv('0 header.csv')

# df_header = df_header[df_header['m_packet_format'] == 2021]
# df_lap_packets = df_header[df_header['m_packet_id'] == 2]
# df_car_telem = df_header[df_header['m_packet_id'] == 6]

,m_packet_format,m_game_major_version,m_game_minor_version,m_packet_version,m_packet_id,m_session_uid,m_session_time,m_frame_identifier,m_player_car_index,m_secondary_player_car_index
0,2021,1,18,1,0,18401439455031850975,2.163044e-41,4278190080,251,32
1,2021,1,18,1,0,558740681343367135,1.136789e-40,4278190080,251,32
2,2021,1,18,1,0,18068454557583144927,2.056644e-40,4278190080,251,32
3,2021,1,18,1,0,8409640731756984287,2.976092e-40,4278190080,251,32
4,2021,1,18,1,0,14838810674805077983,3.895231e-40,4278190080,251,32
...,...,...,...,...,...,...,...,...,...,...
12561,2021,1,18,1,0,7904674623538067423,5.958968e-03,4278190080,250,153
12562,2021,1,18,1,0,8251170319868884959,6.020003e-03,4278190080,21,145
12563,2021,1,18,1,0,8603858465687336927,6.050521e-03,4278190080,16,136
12564,2021,1,18,1,0,8966398235690661855,6.081038e-03,4278190080,204,126


In [14]:
# LAP DATA LOOP
df_lap_data = pd.DataFrame()

ls_m_last_lap_time_in_ms = []
ls_m_current_lap_time_in_ms = []
ls_m_sector1_time_in_ms = []
ls_m_sector2_time_in_ms = []
ls_m_lap_distance = []
ls_m_total_distance = []
ls_m_safety_car_delta = []
ls_m_car_position = []
ls_m_current_lap_num = []
ls_m_pit_status = []
ls_m_num_pit_stops = []
ls_m_sector = []
ls_m_current_lap_invalid = []
ls_m_penalties = []
ls_m_warnings = []
ls_m_num_unserved_drive_through_pens = []
ls_m_num_unserved_stop_go_pens = []
ls_m_grid_position = []
ls_m_driver_status = []
ls_m_result_status = []
ls_m_pit_lane_timer_active = []
ls_m_pit_lane_time_in_lane_in_ms = []
ls_m_pit_stop_timer_in_ms = []
ls_m_pit_stop_should_serve_pen = []

for i in df_lap_packets.index:
        
    d = data[i]
    m_lapdata = LapData.from_buffer_copy(d[24:])
    
    m_last_lap_time_in_ms = m_lapdata.m_last_lap_time_in_ms                                          # Last lap time in milliseconds
    m_current_lap_time_in_ms = m_lapdata.m_current_lap_time_in_ms                                    # Current time around the lap in milliseconds
    m_sector1_time_in_ms = m_lapdata.m_sector1_time_in_ms                                            # Sector 1 time in milliseconds
    m_sector2_time_in_ms = m_lapdata.m_sector2_time_in_ms                                            # Sector 2 time in milliseconds
    m_lap_distance = m_lapdata.m_lap_distance                                                        # Distance vehicle is around current lap in metres – could        # be negative if line hasn’t been crossed yet
    m_total_distance = m_lapdata.m_total_distance                                                    # Total distance travelled in session in metres – could        # be negative if line hasn’t been crossed yet
    m_safety_car_delta = m_lapdata.m_safety_car_delta                                                # Delta in seconds for safety car
    m_car_position = m_lapdata.m_car_position                                                        # Car race position
    m_current_lap_num = m_lapdata.m_current_lap_num                                                  # Current lap number
    m_pit_status = m_lapdata.m_pit_status                                                            # 0 = none, 1 = pitting, 2 = in pit area
    m_num_pit_stops = m_lapdata.m_num_pit_stops                                                      # Number of pit stops taken in this race
    m_sector = m_lapdata.m_sector                                                                    # 0 = sector1, 1 = sector2, 2 = sector3
    m_current_lap_invalid = m_lapdata.m_current_lap_invalid                                          # Current lap invalid - 0 = valid, 1 = invalid
    m_penalties = m_lapdata.m_penalties                                                              # Accumulated time penalties in seconds to be added
    m_warnings = m_lapdata.m_warnings                                                                # Accumulated number of warnings issued
    m_num_unserved_drive_through_pens = m_lapdata.m_num_unserved_drive_through_pens                  # Num drive through pens left to serve
    m_num_unserved_stop_go_pens = m_lapdata.m_num_unserved_stop_go_pens                              # Num stop go pens left to serve
    m_grid_position = m_lapdata.m_grid_position                                                      # Grid position the vehicle started the race in
    m_driver_status = m_lapdata.m_driver_status                                                      # Status of driver - 0 = in garage, 1 = flying lap        # 2 = in lap, 3 = out lap, 4 = on track
    m_result_status = m_lapdata.m_result_status                                                      # Result status - 0 = invalid, 1 = inactive, 2 = active        # 3 = finished, 4 = didnotfinish, 5 = disqualified        # 6 = not classified, 7 = retired
    m_pit_lane_timer_active = m_lapdata.m_pit_lane_timer_active                                      # Pit lane timing, 0 = inactive, 1 = active
    m_pit_lane_time_in_lane_in_ms = m_lapdata.m_pit_lane_time_in_lane_in_ms                          # If active, the current time spent in the pit lane in ms
    m_pit_stop_timer_in_ms = m_lapdata.m_pit_stop_timer_in_ms                                        # Time of the actual pit stop in ms
    m_pit_stop_should_serve_pen = m_lapdata.m_pit_stop_should_serve_pen                              # Whether the car should serve a penalty at this stop
    
    ls_m_last_lap_time_in_ms.append(m_last_lap_time_in_ms)
    ls_m_current_lap_time_in_ms.append(m_current_lap_time_in_ms)
    ls_m_sector1_time_in_ms.append(m_sector1_time_in_ms)
    ls_m_sector2_time_in_ms.append(m_sector2_time_in_ms)
    ls_m_lap_distance.append(m_lap_distance)
    ls_m_total_distance.append(m_total_distance)
    ls_m_safety_car_delta.append(m_safety_car_delta)
    ls_m_car_position.append(m_car_position)
    ls_m_current_lap_num.append(m_current_lap_num)
    ls_m_pit_status.append(m_pit_status)
    ls_m_num_pit_stops.append(m_num_pit_stops)
    ls_m_sector.append(m_sector)
    ls_m_current_lap_invalid.append(m_current_lap_invalid)
    ls_m_penalties.append(m_penalties)
    ls_m_warnings.append(m_warnings)
    ls_m_num_unserved_drive_through_pens.append(m_num_unserved_drive_through_pens)
    ls_m_num_unserved_stop_go_pens.append(m_num_unserved_stop_go_pens)
    ls_m_grid_position.append(m_grid_position)
    ls_m_driver_status.append(m_driver_status)
    ls_m_result_status.append(m_result_status)
    ls_m_pit_lane_timer_active.append(m_pit_lane_timer_active)
    ls_m_pit_lane_time_in_lane_in_ms.append(m_pit_lane_time_in_lane_in_ms)
    ls_m_pit_stop_timer_in_ms.append(m_pit_stop_timer_in_ms)
    ls_m_pit_stop_should_serve_pen.append(m_pit_stop_should_serve_pen)
    
   
df_lap_data['m_last_lap_time_in_ms'] = ls_m_last_lap_time_in_ms
df_lap_data['m_current_lap_time_in_ms'] = ls_m_current_lap_time_in_ms
df_lap_data['m_sector1_time_in_ms'] = ls_m_sector1_time_in_ms
df_lap_data['m_sector2_time_in_ms'] = ls_m_sector2_time_in_ms
df_lap_data['m_lap_distance'] = ls_m_lap_distance
df_lap_data['m_total_distance'] = ls_m_total_distance
df_lap_data['m_safety_car_delta'] = ls_m_safety_car_delta
df_lap_data['m_car_position'] = ls_m_car_position
df_lap_data['m_current_lap_num'] = ls_m_current_lap_num
df_lap_data['m_pit_status'] = ls_m_pit_status
df_lap_data['m_num_pit_stops'] = ls_m_num_pit_stops
df_lap_data['m_sector'] = ls_m_sector
df_lap_data['m_current_lap_invalid'] = ls_m_current_lap_invalid
df_lap_data['m_penalties'] = ls_m_penalties
df_lap_data['m_warnings'] = ls_m_warnings
df_lap_data['m_num_unserved_drive_through_pens'] = ls_m_num_unserved_drive_through_pens
df_lap_data['m_num_unserved_stop_go_pens'] = ls_m_num_unserved_stop_go_pens
df_lap_data['m_grid_position'] = ls_m_grid_position
df_lap_data['m_driver_status'] = ls_m_driver_status
df_lap_data['m_result_status'] = ls_m_result_status
df_lap_data['m_pit_lane_timer_active'] = ls_m_pit_lane_timer_active
df_lap_data['m_pit_lane_time_in_lane_in_ms'] = ls_m_pit_lane_time_in_lane_in_ms
df_lap_data['m_pit_stop_timer_in_ms'] = ls_m_pit_stop_timer_in_ms
df_lap_data['m_pit_stop_should_serve_pen'] = ls_m_pit_stop_should_serve_pen

df_lap_data.to_csv('lap_data.csv')
    

In [33]:
# CAR TELEMETRY LOOP
for k in set(df_header['m_packet_id']):
    try:
        df_car_telem = df_header[df_header['m_packet_id'] == k]
        df_car_telem_data = pd.DataFrame()

        ls_m_speed = []
        ls_m_throttle = []
        ls_m_steer = []
        ls_m_brake = []
        ls_m_clutch = []
        ls_m_gear = []
        ls_m_engine_rpm = []
        ls_m_drs = []
        ls_m_rev_lights_percent = []
        ls_m_rev_lights_bit_value = []
        ls_m_brakes_temperature = []
        ls_m_tyres_surface_temperature = []
        ls_m_tyres_inner_temperature = []
        ls_m_engine_temperature = []
        ls_m_tyres_pressure = []
        ls_m_surface_type = []

        for i in df_car_telem.index:
            
            d = data[i]

            
            m_cartelem = CarTelemetryData.from_buffer_copy(d[24:])

            m_speed = m_cartelem.m_speed                                                                    # Speed of car in kilometres per hour
            m_throttle = m_cartelem.m_throttle                                                              # Amount of throttle applied (0.0 to 1.0)
            m_steer = m_cartelem.m_steer                                                                    # Steering (-1.0 (full lock left) to 1.0 (full lock right))
            m_brake = m_cartelem.m_brake                                                                    # Amount of brake applied (0.0 to 1.0)
            m_clutch = m_cartelem.m_clutch                                                                  # Amount of clutch applied (0 to 100)
            m_gear = m_cartelem.m_gear                                                                      # Gear selected (1-8, N=0, R=-1)
            m_engine_rpm = m_cartelem.m_engine_rpm                                                          # Engine RPM
            m_drs = m_cartelem.m_drs                                                                        # 0 = off, 1 = on
            m_rev_lights_percent = m_cartelem.m_rev_lights_percent                                          # Rev lights indicator (percentage)
            m_rev_lights_bit_value = m_cartelem.m_rev_lights_bit_value                                      # Rev lights (bit 0 = leftmost LED, bit 14 = rightmost LED)
            m_brakes_temperature = m_cartelem.m_brakes_temperature                                          # Brakes temperature (celsius)
            m_tyres_surface_temperature = m_cartelem.m_tyres_surface_temperature                            # Tyres surface temperature (celsius)
            m_tyres_inner_temperature = m_cartelem.m_tyres_inner_temperature                                # Tyres inner temperature (celsius)
            m_engine_temperature = m_cartelem.m_engine_temperature                                          # Engine temperature (celsius)
            m_tyres_pressure = m_cartelem.m_tyres_pressure                                                  # Tyres pressure (PSI)
            m_surface_type = m_cartelem.m_surface_type                                                      # Driving surface, see appendices
            

            ls_m_speed.append(m_speed)
            ls_m_throttle.append(m_throttle)
            ls_m_steer.append(m_steer)
            ls_m_brake.append(m_brake)
            ls_m_clutch.append(m_clutch)
            ls_m_gear.append(m_gear)
            ls_m_engine_rpm.append(m_engine_rpm)
            ls_m_drs.append(m_drs)
            ls_m_rev_lights_percent.append(m_rev_lights_percent)
            ls_m_rev_lights_bit_value.append(m_rev_lights_bit_value)
            ls_m_brakes_temperature.append(m_brakes_temperature)
            ls_m_tyres_surface_temperature.append(m_tyres_surface_temperature)
            ls_m_tyres_inner_temperature.append(m_tyres_inner_temperature)
            ls_m_engine_temperature.append(m_engine_temperature)
            ls_m_tyres_pressure.append(m_tyres_pressure)
            ls_m_surface_type.append(m_surface_type)
            

        df_car_telem_data['m_speed'] = ls_m_speed
        df_car_telem_data['m_throttle'] = ls_m_throttle
        df_car_telem_data['m_steer'] = ls_m_steer
        df_car_telem_data['m_brake'] = ls_m_brake
        df_car_telem_data['m_clutch'] = ls_m_clutch
        df_car_telem_data['m_gear'] = ls_m_gear
        df_car_telem_data['m_engine_rpm'] = ls_m_engine_rpm
        df_car_telem_data['m_drs'] = ls_m_drs
        df_car_telem_data['m_rev_lights_percent'] = ls_m_rev_lights_percent
        df_car_telem_data['m_rev_lights_bit_value'] = ls_m_rev_lights_bit_value
        df_car_telem_data['m_brakes_temperature'] = ls_m_brakes_temperature
        df_car_telem_data['m_tyres_surface_temperature'] = ls_m_tyres_surface_temperature
        df_car_telem_data['m_tyres_inner_temperature'] = ls_m_tyres_inner_temperature
        df_car_telem_data['m_engine_temperature'] = ls_m_engine_temperature
        df_car_telem_data['m_tyres_pressure'] = ls_m_tyres_pressure
        df_car_telem_data['m_surface_type'] = ls_m_surface_type


        df_car_telem_data.to_csv(f'car telemetry {k}.csv')
        
    except:
        pass

In [18]:
# CAR MOTION LOOP
for k in set(df_header['m_packet_id']):
    try:
        df_car_motion = df_header[df_header['m_packet_id'] == k]
        df_car_motion_data = pd.DataFrame()

        ls_m_world_position_x = []
        ls_m_world_position_y = []
        ls_m_world_position_z = []
        ls_m_world_velocity_x = []
        ls_m_world_velocity_y = []
        ls_m_world_velocity_z = []
        ls_m_world_forward_dir_x = []
        ls_m_world_forward_dir_y = []
        ls_m_world_forward_dir_z = []
        ls_m_world_right_dir_x = []
        ls_m_world_right_dir_y = []
        ls_m_world_right_dir_z = []
        ls_m_g_force_lateral = []
        ls_m_g_force_longitudinal = []
        ls_m_g_force_vertical = []
        ls_m_yaw = []
        ls_m_pitch = []
        ls_m_roll = []

        for i in df_car_motion.index:
            
            d = data[i]
            

            m_motion = CarMotionData.from_buffer_copy(d[24:])
            
            m_world_position_x = m_motion.m_world_position_x
            m_world_position_y = m_motion.m_world_position_y
            m_world_position_z = m_motion.m_world_position_z
            m_world_velocity_x = m_motion.m_world_velocity_x
            m_world_velocity_y = m_motion.m_world_velocity_y
            m_world_velocity_z = m_motion.m_world_velocity_z
            m_world_forward_dir_x = m_motion.m_world_forward_dir_x
            m_world_forward_dir_y = m_motion.m_world_forward_dir_y
            m_world_forward_dir_z = m_motion.m_world_forward_dir_z
            m_world_right_dir_x = m_motion.m_world_right_dir_x
            m_world_right_dir_y = m_motion.m_world_right_dir_y
            m_world_right_dir_z = m_motion.m_world_right_dir_z
            m_g_force_lateral = m_motion.m_g_force_lateral
            m_g_force_longitudinal = m_motion.m_g_force_longitudinal
            m_g_force_vertical = m_motion.m_g_force_vertical
            m_yaw = m_motion.m_yaw
            m_pitch = m_motion.m_pitch
            m_roll = m_motion.m_roll
            
            ls_m_world_position_x.append(m_world_position_x)
            ls_m_world_position_y.append(m_world_position_y)
            ls_m_world_position_z.append(m_world_position_z)
            ls_m_world_velocity_x.append(m_world_velocity_x)
            ls_m_world_velocity_y.append(m_world_velocity_y)
            ls_m_world_velocity_z.append(m_world_velocity_z)
            ls_m_world_forward_dir_x.append(m_world_forward_dir_x)
            ls_m_world_forward_dir_y.append(m_world_forward_dir_y)
            ls_m_world_forward_dir_z.append(m_world_forward_dir_z)
            ls_m_world_right_dir_x.append(m_world_right_dir_x)
            ls_m_world_right_dir_y.append(m_world_right_dir_y)
            ls_m_world_right_dir_z.append(m_world_right_dir_z)
            ls_m_g_force_lateral.append(m_g_force_lateral)
            ls_m_g_force_longitudinal.append(m_g_force_longitudinal)
            ls_m_g_force_vertical.append(m_g_force_vertical)
            ls_m_yaw.append(m_yaw)
            ls_m_pitch.append(m_pitch)
            ls_m_roll.append(m_roll)
            
        
        df_car_motion_data['m_world_position_x'] = ls_m_world_position_x
        df_car_motion_data['m_world_position_y'] = ls_m_world_position_y
        df_car_motion_data['m_world_position_z'] = ls_m_world_position_z
        df_car_motion_data['m_world_velocity_x'] = ls_m_world_velocity_x
        df_car_motion_data['m_world_velocity_y'] = ls_m_world_velocity_y
        df_car_motion_data['m_world_velocity_z'] = ls_m_world_velocity_z
        df_car_motion_data['m_world_forward_dir_x'] = ls_m_world_forward_dir_x
        df_car_motion_data['m_world_forward_dir_y'] = ls_m_world_forward_dir_y
        df_car_motion_data['m_world_forward_dir_z'] = ls_m_world_forward_dir_z
        df_car_motion_data['m_world_right_dir_x'] = ls_m_world_right_dir_x
        df_car_motion_data['m_world_right_dir_y'] = ls_m_world_right_dir_y
        df_car_motion_data['m_world_right_dir_z'] = ls_m_world_right_dir_z
        df_car_motion_data['m_g_force_lateral'] = ls_m_g_force_lateral
        df_car_motion_data['m_g_force_longitudinal'] = ls_m_g_force_longitudinal
        df_car_motion_data['m_g_force_vertical'] = ls_m_g_force_vertical
        df_car_motion_data['m_yaw'] = ls_m_yaw
        df_car_motion_data['m_pitch'] = ls_m_pitch
        df_car_motion_data['m_roll'] = ls_m_roll

        df_car_motion_data.to_csv(f'car motion {k}.csv')
        
    except Exception as e:
        print(e)
        pass

In [38]:
# SESSION DATA LOOP

# CAR MOTION LOOP
for k in set(df_header['m_packet_id']):
    try:
        
        df_session = df_header[df_header['m_packet_id'] == k]

        ls_m_header = []
        ls_m_weather = []
        ls_m_track_temperature = []
        ls_m_air_temperature = []
        ls_m_total_laps = []
        ls_m_track_length = []
        ls_m_session_type = []
        ls_m_track_id = []
        ls_m_formula = []
        ls_m_session_time_left = []
        ls_m_session_duration = []
        ls_m_pit_speed_limit = []
        ls_m_game_paused = []
        ls_m_is_spectating = []
        ls_m_spectator_car_index = []
        ls_m_sli_pro_native_support = []
        ls_m_num_marshal_zones = []
        ls_m_marshal_zones = []
        ls_m_safety_car_status = []
        ls_m_network_game = []
        ls_m_num_weather_forecast_samples = []
        ls_m_weather_forecast_samples = []
        ls_m_forecast_accuracy = []
        ls_m_ai_difficulty = []
        ls_m_season_link_identifier = []
        ls_m_weekend_link_identifier = []
        ls_m_session_link_identifier = []
        ls_m_pit_stop_window_ideal_lap = []
        ls_m_pit_stop_window_latest_lap = []
        ls_m_pit_stop_rejoin_position = []
        ls_m_steering_assist = []
        ls_m_braking_assist = []
        ls_m_gearbox_assist = []
        ls_m_pit_assist = []
        ls_m_pit_release_assist = []
        ls_m_ersassist = []
        ls_m_drsassist = []
        ls_m_dynamic_racing_line = []
        ls_m_dynamic_racing_line_type = []


        df_session_data = pd.DataFrame()

        for i in df_session.index:
            
            d = data[i]
            session_data = PacketSessionData.from_buffer_copy(d[24:])
            
            # m_header = session_data.m_header
            m_weather = session_data.m_weather
            m_track_temperature = session_data.m_track_temperature
            m_air_temperature = session_data.m_air_temperature
            m_total_laps = session_data.m_total_laps
            m_track_length = session_data.m_track_length
            m_session_type = session_data.m_session_type
            m_track_id = session_data.m_track_id
            m_formula = session_data.m_formula
            m_session_time_left = session_data.m_session_time_left
            m_session_duration = session_data.m_session_duration
            m_pit_speed_limit = session_data.m_pit_speed_limit
            m_game_paused = session_data.m_game_paused
            m_is_spectating = session_data.m_is_spectating
            m_spectator_car_index = session_data.m_spectator_car_index
            m_sli_pro_native_support = session_data.m_sli_pro_native_support
            m_num_marshal_zones = session_data.m_num_marshal_zones
            m_marshal_zones = session_data.m_marshal_zones
            m_safety_car_status = session_data.m_safety_car_status
            m_network_game = session_data.m_network_game
            m_num_weather_forecast_samples = session_data.m_num_weather_forecast_samples
            m_weather_forecast_samples = session_data.m_weather_forecast_samples
            m_forecast_accuracy = session_data.m_forecast_accuracy
            m_ai_difficulty = session_data.m_ai_difficulty
            m_season_link_identifier = session_data.m_season_link_identifier
            m_weekend_link_identifier = session_data.m_weekend_link_identifier
            m_session_link_identifier = session_data.m_session_link_identifier
            m_pit_stop_window_ideal_lap = session_data.m_pit_stop_window_ideal_lap
            m_pit_stop_window_latest_lap = session_data.m_pit_stop_window_latest_lap
            m_pit_stop_rejoin_position = session_data.m_pit_stop_rejoin_position
            m_steering_assist = session_data.m_steering_assist
            m_braking_assist = session_data.m_braking_assist
            m_gearbox_assist = session_data.m_gearbox_assist
            m_pit_assist = session_data.m_pit_assist
            m_pit_release_assist = session_data.m_pit_release_assist
            m_ersassist = session_data.m_ersassist
            m_drsassist = session_data.m_drsassist
            m_dynamic_racing_line = session_data.m_dynamic_racing_line
            m_dynamic_racing_line_type = session_data.m_dynamic_racing_line_type
            
            # ls_m_header.append(m_header)
            ls_m_weather.append(m_weather)
            ls_m_track_temperature.append(m_track_temperature)
            ls_m_air_temperature.append(m_air_temperature)
            ls_m_total_laps.append(m_total_laps)
            ls_m_track_length.append(m_track_length)
            ls_m_session_type.append(m_session_type)
            ls_m_track_id.append(m_track_id)
            ls_m_formula.append(m_formula)
            ls_m_session_time_left.append(m_session_time_left)
            ls_m_session_duration.append(m_session_duration)
            ls_m_pit_speed_limit.append(m_pit_speed_limit)
            ls_m_game_paused.append(m_game_paused)
            ls_m_is_spectating.append(m_is_spectating)
            ls_m_spectator_car_index.append(m_spectator_car_index)
            ls_m_sli_pro_native_support.append(m_sli_pro_native_support)
            ls_m_num_marshal_zones.append(m_num_marshal_zones)
            ls_m_marshal_zones.append(m_marshal_zones)
            ls_m_safety_car_status.append(m_safety_car_status)
            ls_m_network_game.append(m_network_game)
            ls_m_num_weather_forecast_samples.append(m_num_weather_forecast_samples)
            ls_m_weather_forecast_samples.append(m_weather_forecast_samples)
            ls_m_forecast_accuracy.append(m_forecast_accuracy)
            ls_m_ai_difficulty.append(m_ai_difficulty)
            ls_m_season_link_identifier.append(m_season_link_identifier)
            ls_m_weekend_link_identifier.append(m_weekend_link_identifier)
            ls_m_session_link_identifier.append(m_session_link_identifier)
            ls_m_pit_stop_window_ideal_lap.append(m_pit_stop_window_ideal_lap)
            ls_m_pit_stop_window_latest_lap.append(m_pit_stop_window_latest_lap)
            ls_m_pit_stop_rejoin_position.append(m_pit_stop_rejoin_position)
            ls_m_steering_assist.append(m_steering_assist)
            ls_m_braking_assist.append(m_braking_assist)
            ls_m_gearbox_assist.append(m_gearbox_assist)
            ls_m_pit_assist.append(m_pit_assist)
            ls_m_pit_release_assist.append(m_pit_release_assist)
            ls_m_ersassist.append(m_ersassist)
            ls_m_drsassist.append(m_drsassist)
            ls_m_dynamic_racing_line.append(m_dynamic_racing_line)
            ls_m_dynamic_racing_line_type.append(m_dynamic_racing_line_type)
            
            
        # df_session_data['m_header'] = ls_m_header
        df_session_data['m_weather'] = ls_m_weather
        df_session_data['m_track_temperature'] = ls_m_track_temperature
        df_session_data['m_air_temperature'] = ls_m_air_temperature
        df_session_data['m_total_laps'] = ls_m_total_laps
        df_session_data['m_track_length'] = ls_m_track_length
        df_session_data['m_session_type'] = ls_m_session_type
        df_session_data['m_track_id'] = ls_m_track_id
        df_session_data['m_formula'] = ls_m_formula
        df_session_data['m_session_time_left'] = ls_m_session_time_left
        df_session_data['m_session_duration'] = ls_m_session_duration
        df_session_data['m_pit_speed_limit'] = ls_m_pit_speed_limit
        df_session_data['m_game_paused'] = ls_m_game_paused
        df_session_data['m_is_spectating'] = ls_m_is_spectating
        df_session_data['m_spectator_car_index'] = ls_m_spectator_car_index
        df_session_data['m_sli_pro_native_support'] = ls_m_sli_pro_native_support
        df_session_data['m_num_marshal_zones'] = ls_m_num_marshal_zones
        df_session_data['m_marshal_zones'] = ls_m_marshal_zones
        df_session_data['m_safety_car_status'] = ls_m_safety_car_status
        df_session_data['m_network_game'] = ls_m_network_game
        df_session_data['m_num_weather_forecast_samples'] = ls_m_num_weather_forecast_samples
        df_session_data['m_weather_forecast_samples'] = ls_m_weather_forecast_samples
        df_session_data['m_forecast_accuracy'] = ls_m_forecast_accuracy
        df_session_data['m_ai_difficulty'] = ls_m_ai_difficulty
        df_session_data['m_season_link_identifier'] = ls_m_season_link_identifier
        df_session_data['m_weekend_link_identifier'] = ls_m_weekend_link_identifier
        df_session_data['m_session_link_identifier'] = ls_m_session_link_identifier
        df_session_data['m_pit_stop_window_ideal_lap'] = ls_m_pit_stop_window_ideal_lap
        df_session_data['m_pit_stop_window_latest_lap'] = ls_m_pit_stop_window_latest_lap
        df_session_data['m_pit_stop_rejoin_position'] = ls_m_pit_stop_rejoin_position
        df_session_data['m_steering_assist'] = ls_m_steering_assist
        df_session_data['m_braking_assist'] = ls_m_braking_assist
        df_session_data['m_gearbox_assist'] = ls_m_gearbox_assist
        df_session_data['m_pit_assist'] = ls_m_pit_assist
        df_session_data['m_pit_release_assist'] = ls_m_pit_release_assist
        df_session_data['m_ersassist'] = ls_m_ersassist
        df_session_data['m_drsassist'] = ls_m_drsassist
        df_session_data['m_dynamic_racing_line'] = ls_m_dynamic_racing_line
        df_session_data['m_dynamic_racing_line_type'] = ls_m_dynamic_racing_line_type
        
        df_session_data.to_csv(f'session_data {k}.csv')
        
    except Exception as e:
        print(e)

Buffer size too small (601 instead of at least 704 bytes)


c:\Users\jules\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\dtypes\cast.py:1981: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  result[:] = values


Buffer size too small (12 instead of at least 704 bytes)


c:\Users\jules\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\dtypes\cast.py:1981: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  result[:] = values
c:\Users\jules\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\dtypes\cast.py:1981: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  result[:] = values
c:\Users\jules\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\dtypes\cast.py:1981: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  result[:] = values
c:\Users\jules\AppData\Local\Pr